# Ordering by metrics and retraining phase

## Dataset: CIFAR-10

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
#pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

#from tf.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2

import glob
import os
import cv2
import tensorflow.keras as keras


import time
import argparse

from tqdm import tqdm

from tensorflow.keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd ../utils/

D:\guided-retraining\utils


In [4]:
# utils for project
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "fashion"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train",True)
x_val,y_val = utils.get_data(dataset,"Val",True)
x_test,y_test = utils.get_data(dataset,"Test",True)

D:/guided-retraining/data/fashion/x_train.npy
x_set len:  48999
D:/guided-retraining/data/fashion/y_train.npy
y_set len:  48999
D:/guided-retraining/data/fashion/x_val.npy
x_set len:  14000
D:/guided-retraining/data/fashion/y_val.npy
y_set len:  14000
D:/guided-retraining/data/fashion/x_test.npy
x_set len:  7001
D:/guided-retraining/data/fashion/y_test.npy
y_set len:  7001


In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",True)

D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary.npy
x_set len:  55998
D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary_labels.npy
y_set len:  55998


In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6999
6999


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial',True) 

D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy
x_set len:  14000
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
y_set len:  14000


In [11]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6999
6999


## ----

In [12]:
dataset

'fashion'

In [13]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"
elif(dataset == 'fashion'):
    model_dir = "D:/guided-retraining/models/model_fashion_2"


In [14]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

D:/guided-retraining/models/model_fashion_2
Model loaded correctly


In [15]:
print(x_train_and_adversary.shape)
len(x_train_and_adversary)//20

(55998, 28, 28, 1)


2799

In [17]:
n_data_points = 2800

## Loading LSA and DSA values 

In [17]:
lsa_direction = "D:/guided-retraining/data/"+dataset+"/lsa_values.npy"

In [18]:
lsa_values = np.load(lsa_direction)


In [19]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [20]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [21]:
len(image_sets_lsa[-1])

55998

In [22]:
len(image_sets_lsa)

20

## Training guided by LSA

In [23]:
len(label_sets_lsa)

20

In [24]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_lsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [25]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n=n+1

Epoch 1/10
44/44 [==============================] - 6s 121ms/step - loss: 1.1453 - accuracy: 0.6325 - val_loss: 1.8995 - val_accuracy: 0.2497
Epoch 2/10
44/44 [==============================] - 5s 120ms/step - loss: 0.6790 - accuracy: 0.7607 - val_loss: 1.5452 - val_accuracy: 0.4357
Epoch 3/10
44/44 [==============================] - 5s 117ms/step - loss: 0.5767 - accuracy: 0.8093 - val_loss: 1.4715 - val_accuracy: 0.3969
Epoch 4/10
44/44 [==============================] - 5s 120ms/step - loss: 0.5099 - accuracy: 0.8325 - val_loss: 1.2761 - val_accuracy: 0.4926
Epoch 5/10
44/44 [==============================] - 5s 119ms/step - loss: 0.4569 - accuracy: 0.8507 - val_loss: 1.2127 - val_accuracy: 0.5292
Epoch 6/10
44/44 [==============================] - 5s 121ms/step - loss: 0.4175 - accuracy: 0.8686 - val_loss: 1.2049 - val_accuracy: 0.5411
Epoch 7/10
44/44 [==============================] - 6s 131ms/step - loss: 0.3992 - accuracy: 0.8732 - val_loss: 1.0491 - val_accuracy: 0.5804
Epoch 

In [26]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

Epoch 1/10
88/88 [==============================] - 10s 102ms/step - loss: 0.6502 - accuracy: 0.8209 - val_loss: 2.2610 - val_accuracy: 0.1167
Epoch 2/10
88/88 [==============================] - 8s 97ms/step - loss: 0.3499 - accuracy: 0.8914 - val_loss: 2.0406 - val_accuracy: 0.3086
Epoch 3/10
88/88 [==============================] - 8s 93ms/step - loss: 0.2875 - accuracy: 0.9182 - val_loss: 1.7993 - val_accuracy: 0.3264
Epoch 4/10
88/88 [==============================] - 8s 94ms/step - loss: 0.2419 - accuracy: 0.9268 - val_loss: 1.5435 - val_accuracy: 0.3779
Epoch 5/10
88/88 [==============================] - 8s 95ms/step - loss: 0.2319 - accuracy: 0.9334 - val_loss: 1.4411 - val_accuracy: 0.4149
Epoch 6/10
88/88 [==============================] - 8s 95ms/step - loss: 0.2071 - accuracy: 0.9379 - val_loss: 1.4150 - val_accuracy: 0.4315
Epoch 7/10
88/88 [==============================] - 8s 96ms/step - loss: 0.1891 - accuracy: 0.9443 - val_loss: 1.2942 - val_accuracy: 0.4514
Epoch 8/10


In [27]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 12s 88ms/step - loss: 0.5751 - accuracy: 0.8293 - val_loss: 1.9227 - val_accuracy: 0.3011
Epoch 2/10
132/132 [==============================] - 12s 91ms/step - loss: 0.3214 - accuracy: 0.9004 - val_loss: 2.0457 - val_accuracy: 0.3510
Epoch 3/10
132/132 [==============================] - 12s 90ms/step - loss: 0.2559 - accuracy: 0.9251 - val_loss: 1.7393 - val_accuracy: 0.3885
Epoch 4/10
132/132 [==============================] - 11s 87ms/step - loss: 0.2214 - accuracy: 0.9335 - val_loss: 1.4281 - val_accuracy: 0.4799
Epoch 5/10
132/132 [==============================] - 12s 90ms/step - loss: 0.1961 - accuracy: 0.9421 - val_loss: 1.3229 - val_accuracy: 0.4995
Epoch 6/10
132/132 [==============================] - 11s 86ms/step - loss: 0.1793 - accuracy: 0.9473 - val_loss: 1.2338 - val_accuracy: 0.5113
Epoch 7/10
132/132 [==============================] - 12s 93ms/step - loss: 0.1628 - accuracy: 0.9532 - val_loss: 1.4290 - val_accurac

In [28]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 15s 79ms/step - loss: 0.4625 - accuracy: 0.8601 - val_loss: 1.8190 - val_accuracy: 0.3098
Epoch 2/10
175/175 [==============================] - 14s 80ms/step - loss: 0.2423 - accuracy: 0.9309 - val_loss: 1.5366 - val_accuracy: 0.4510
Epoch 3/10
175/175 [==============================] - 14s 81ms/step - loss: 0.1961 - accuracy: 0.9470 - val_loss: 1.6069 - val_accuracy: 0.4362
Epoch 4/10
175/175 [==============================] - 14s 81ms/step - loss: 0.1715 - accuracy: 0.9511 - val_loss: 1.5012 - val_accuracy: 0.4571
Epoch 5/10
175/175 [==============================] - 14s 82ms/step - loss: 0.1511 - accuracy: 0.9595 - val_loss: 1.3051 - val_accuracy: 0.5050
Epoch 6/10
175/175 [==============================] - 14s 81ms/step - loss: 0.1391 - accuracy: 0.9602 - val_loss: 1.2415 - val_accuracy: 0.5133
Epoch 7/10
175/175 [==============================] - 14s 81ms/step - loss: 0.1296 - accuracy: 0.9638 - val_loss: 1.3769 - val_accurac

In [29]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 18s 77ms/step - loss: 0.5881 - accuracy: 0.8365 - val_loss: 1.6930 - val_accuracy: 0.3957
Epoch 2/10
219/219 [==============================] - 17s 78ms/step - loss: 0.3484 - accuracy: 0.9054 - val_loss: 1.6368 - val_accuracy: 0.4396
Epoch 3/10
219/219 [==============================] - 18s 80ms/step - loss: 0.2985 - accuracy: 0.9174 - val_loss: 1.3664 - val_accuracy: 0.4990
Epoch 4/10
219/219 [==============================] - 19s 87ms/step - loss: 0.2599 - accuracy: 0.9254 - val_loss: 1.2926 - val_accuracy: 0.5331
Epoch 5/10
219/219 [==============================] - 20s 91ms/step - loss: 0.2453 - accuracy: 0.9296 - val_loss: 1.2990 - val_accuracy: 0.5061
Epoch 6/10
219/219 [==============================] - 19s 87ms/step - loss: 0.2345 - accuracy: 0.9313 - val_loss: 1.1300 - val_accuracy: 0.5672
Epoch 7/10
219/219 [==============================] - 18s 82ms/step - loss: 0.2198 - accuracy: 0.9361 - val_loss: 1.1572 - val_accurac

In [30]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 21s 76ms/step - loss: 0.5555 - accuracy: 0.8623 - val_loss: 1.9241 - val_accuracy: 0.3872
Epoch 2/10
263/263 [==============================] - 20s 76ms/step - loss: 0.3357 - accuracy: 0.9137 - val_loss: 1.5666 - val_accuracy: 0.4426
Epoch 3/10
263/263 [==============================] - 20s 77ms/step - loss: 0.2825 - accuracy: 0.9264 - val_loss: 1.4229 - val_accuracy: 0.5259
Epoch 4/10
263/263 [==============================] - 20s 76ms/step - loss: 0.2632 - accuracy: 0.9305 - val_loss: 1.2935 - val_accuracy: 0.5459
Epoch 5/10
263/263 [==============================] - 21s 81ms/step - loss: 0.2437 - accuracy: 0.9353 - val_loss: 1.1506 - val_accuracy: 0.5473
Epoch 6/10
263/263 [==============================] - 27s 103ms/step - loss: 0.2380 - accuracy: 0.9371 - val_loss: 1.4674 - val_accuracy: 0.5284
Epoch 7/10
263/263 [==============================] - 21s 81ms/step - loss: 0.2281 - accuracy: 0.9382 - val_loss: 1.1205 - val_accura

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 23s 74ms/step - loss: 0.6067 - accuracy: 0.8281 - val_loss: 1.7611 - val_accuracy: 0.4636
Epoch 2/10
307/307 [==============================] - 23s 74ms/step - loss: 0.3456 - accuracy: 0.9068 - val_loss: 1.2362 - val_accuracy: 0.5479
Epoch 3/10
307/307 [==============================] - 23s 74ms/step - loss: 0.2957 - accuracy: 0.9202 - val_loss: 1.1722 - val_accuracy: 0.5665
Epoch 4/10
307/307 [==============================] - 23s 74ms/step - loss: 0.2688 - accuracy: 0.9251 - val_loss: 1.5242 - val_accuracy: 0.5582
Epoch 5/10
307/307 [==============================] - 23s 76ms/step - loss: 0.2588 - accuracy: 0.9281 - val_loss: 1.1525 - val_accuracy: 0.5871
Epoch 6/10
307/307 [==============================] - 23s 73ms/step - loss: 0.2406 - accuracy: 0.9324 - val_loss: 1.3167 - val_accuracy: 0.5799
Epoch 7/10
307/307 [==============================] - 23s 73ms/step - loss: 0.2330 - accuracy: 0.9352 - val_loss: 1.2196 - val_accurac

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 26s 72ms/step - loss: 0.5927 - accuracy: 0.8271 - val_loss: 1.9035 - val_accuracy: 0.4774
Epoch 2/10
350/350 [==============================] - 25s 71ms/step - loss: 0.3371 - accuracy: 0.9107 - val_loss: 1.3978 - val_accuracy: 0.5378
Epoch 3/10
350/350 [==============================] - 25s 72ms/step - loss: 0.2818 - accuracy: 0.9255 - val_loss: 1.3106 - val_accuracy: 0.5570
Epoch 4/10
350/350 [==============================] - 25s 71ms/step - loss: 0.2573 - accuracy: 0.9297 - val_loss: 1.4116 - val_accuracy: 0.5795
Epoch 5/10
350/350 [==============================] - 25s 71ms/step - loss: 0.2478 - accuracy: 0.9321 - val_loss: 1.0586 - val_accuracy: 0.6021
Epoch 6/10
350/350 [==============================] - 25s 71ms/step - loss: 0.2354 - accuracy: 0.9346 - val_loss: 1.0367 - val_accuracy: 0.6069
Epoch 7/10
350/350 [==============================] - 25s 71ms/step - loss: 0.2331 - accuracy: 0.9362 - val_loss: 1.0105 - val_accurac

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 29s 71ms/step - loss: 0.5571 - accuracy: 0.8387 - val_loss: 1.5977 - val_accuracy: 0.4784
Epoch 2/10
394/394 [==============================] - 28s 71ms/step - loss: 0.3021 - accuracy: 0.9218 - val_loss: 1.4667 - val_accuracy: 0.5331
Epoch 3/10
394/394 [==============================] - 28s 71ms/step - loss: 0.2506 - accuracy: 0.9339 - val_loss: 1.2457 - val_accuracy: 0.5576
Epoch 4/10
394/394 [==============================] - 28s 71ms/step - loss: 0.2357 - accuracy: 0.9378 - val_loss: 1.1268 - val_accuracy: 0.5714
Epoch 5/10
394/394 [==============================] - 28s 72ms/step - loss: 0.2244 - accuracy: 0.9394 - val_loss: 1.2313 - val_accuracy: 0.5611
Epoch 6/10
394/394 [==============================] - 28s 71ms/step - loss: 0.2206 - accuracy: 0.9417 - val_loss: 1.1001 - val_accuracy: 0.5797
Epoch 7/10
394/394 [==============================] - 28s 71ms/step - loss: 0.2097 - accuracy: 0.9429 - val_loss: 1.0530 - val_accurac

In [34]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 32s 70ms/step - loss: 0.5255 - accuracy: 0.8521 - val_loss: 1.4894 - val_accuracy: 0.4832
Epoch 2/10
438/438 [==============================] - 31s 70ms/step - loss: 0.2798 - accuracy: 0.9292 - val_loss: 1.3062 - val_accuracy: 0.5343
Epoch 3/10
438/438 [==============================] - 261s 596ms/step - loss: 0.2398 - accuracy: 0.9375 - val_loss: 1.3707 - val_accuracy: 0.5762
Epoch 4/10
438/438 [==============================] - 31s 72ms/step - loss: 0.2257 - accuracy: 0.9428 - val_loss: 1.1767 - val_accuracy: 0.5815
Epoch 5/10
438/438 [==============================] - 27s 61ms/step - loss: 0.2146 - accuracy: 0.9446 - val_loss: 1.0815 - val_accuracy: 0.5932
Epoch 6/10
438/438 [==============================] - 27s 61ms/step - loss: 0.2083 - accuracy: 0.9454 - val_loss: 1.1912 - val_accuracy: 0.5951
Epoch 7/10
438/438 [==============================] - 28s 63ms/step - loss: 0.2083 - accuracy: 0.9453 - val_loss: 1.2714 - val_accur

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 31s 63ms/step - loss: 0.5651 - accuracy: 0.8395 - val_loss: 1.3847 - val_accuracy: 0.5624
Epoch 2/10
482/482 [==============================] - 30s 62ms/step - loss: 0.3192 - accuracy: 0.9126 - val_loss: 0.9948 - val_accuracy: 0.5707
Epoch 3/10
482/482 [==============================] - 30s 62ms/step - loss: 0.2813 - accuracy: 0.9204 - val_loss: 1.2220 - val_accuracy: 0.5737
Epoch 4/10
482/482 [==============================] - 30s 62ms/step - loss: 0.2568 - accuracy: 0.9254 - val_loss: 1.1455 - val_accuracy: 0.5776
Epoch 5/10
482/482 [==============================] - 30s 61ms/step - loss: 0.2447 - accuracy: 0.9283 - val_loss: 0.9489 - val_accuracy: 0.6184
Epoch 6/10
482/482 [==============================] - 30s 61ms/step - loss: 0.2386 - accuracy: 0.9317 - val_loss: 0.9590 - val_accuracy: 0.5856
Epoch 7/10
482/482 [==============================] - 30s 61ms/step - loss: 0.2405 - accuracy: 0.9331 - val_loss: 1.1464 - val_accura

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 33s 62ms/step - loss: 0.5886 - accuracy: 0.8311 - val_loss: 1.3274 - val_accuracy: 0.5676
Epoch 2/10
525/525 [==============================] - 33s 62ms/step - loss: 0.3339 - accuracy: 0.9111 - val_loss: 1.2207 - val_accuracy: 0.5720
Epoch 3/10
525/525 [==============================] - 33s 64ms/step - loss: 0.2930 - accuracy: 0.9197 - val_loss: 1.1146 - val_accuracy: 0.5746
Epoch 4/10
525/525 [==============================] - 34s 64ms/step - loss: 0.2757 - accuracy: 0.9232 - val_loss: 1.1016 - val_accuracy: 0.5768
Epoch 5/10
525/525 [==============================] - 34s 65ms/step - loss: 0.2686 - accuracy: 0.9261 - val_loss: 1.2227 - val_accuracy: 0.5777
Epoch 6/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2580 - accuracy: 0.9277 - val_loss: 1.3556 - val_accuracy: 0.6002
Epoch 7/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2578 - accuracy: 0.9307 - val_loss: 1.0696 - val_accura

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 41s 71ms/step - loss: 0.6537 - accuracy: 0.7980 - val_loss: 1.3016 - val_accuracy: 0.5955
Epoch 2/10
569/569 [==============================] - 40s 70ms/step - loss: 0.4036 - accuracy: 0.8818 - val_loss: 0.9973 - val_accuracy: 0.6078
Epoch 3/10
569/569 [==============================] - 39s 69ms/step - loss: 0.3548 - accuracy: 0.8923 - val_loss: 0.8514 - val_accuracy: 0.6344
Epoch 4/10
569/569 [==============================] - 39s 69ms/step - loss: 0.3304 - accuracy: 0.8984 - val_loss: 0.9412 - val_accuracy: 0.6440
Epoch 5/10
569/569 [==============================] - 39s 69ms/step - loss: 0.3184 - accuracy: 0.9027 - val_loss: 0.9720 - val_accuracy: 0.6458
Epoch 6/10
569/569 [==============================] - 39s 69ms/step - loss: 0.3111 - accuracy: 0.9045 - val_loss: 1.0099 - val_accuracy: 0.6274
Epoch 7/10
569/569 [==============================] - 40s 70ms/step - loss: 0.3115 - accuracy: 0.9061 - val_loss: 0.8648 - val_accura

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 42s 68ms/step - loss: 0.6760 - accuracy: 0.7864 - val_loss: 1.0099 - val_accuracy: 0.6289
Epoch 2/10
613/613 [==============================] - 42s 68ms/step - loss: 0.4192 - accuracy: 0.8743 - val_loss: 0.8760 - val_accuracy: 0.6421
Epoch 3/10
613/613 [==============================] - 41s 68ms/step - loss: 0.3625 - accuracy: 0.8896 - val_loss: 0.8179 - val_accuracy: 0.6530
Epoch 4/10
613/613 [==============================] - 41s 67ms/step - loss: 0.3435 - accuracy: 0.8965 - val_loss: 0.9336 - val_accuracy: 0.6529
Epoch 5/10
613/613 [==============================] - 42s 68ms/step - loss: 0.3307 - accuracy: 0.9010 - val_loss: 0.7191 - val_accuracy: 0.6551
Epoch 6/10
613/613 [==============================] - 41s 67ms/step - loss: 0.3210 - accuracy: 0.9050 - val_loss: 0.7941 - val_accuracy: 0.6596
Epoch 7/10
613/613 [==============================] - 42s 69ms/step - loss: 0.3163 - accuracy: 0.9058 - val_loss: 1.1358 - val_accura

In [39]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 52s 78ms/step - loss: 0.7616 - accuracy: 0.7421 - val_loss: 0.9044 - val_accuracy: 0.6905
Epoch 2/10
657/657 [==============================] - 50s 76ms/step - loss: 0.4756 - accuracy: 0.8432 - val_loss: 0.8097 - val_accuracy: 0.7167
Epoch 3/10
657/657 [==============================] - 51s 78ms/step - loss: 0.4168 - accuracy: 0.8659 - val_loss: 0.7265 - val_accuracy: 0.7201
Epoch 4/10
657/657 [==============================] - 50s 76ms/step - loss: 0.3885 - accuracy: 0.8740 - val_loss: 0.7810 - val_accuracy: 0.7312
Epoch 5/10
657/657 [==============================] - 50s 76ms/step - loss: 0.3648 - accuracy: 0.8822 - val_loss: 0.6732 - val_accuracy: 0.7276
Epoch 6/10
657/657 [==============================] - 50s 77ms/step - loss: 0.3598 - accuracy: 0.8841 - val_loss: 0.6091 - val_accuracy: 0.7439
Epoch 7/10
657/657 [==============================] - 50s 76ms/step - loss: 0.3551 - accuracy: 0.8885 - val_loss: 0.8819 - val_accura

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 53s 75ms/step - loss: 0.7527 - accuracy: 0.7439 - val_loss: 0.7593 - val_accuracy: 0.6878
Epoch 2/10
700/700 [==============================] - 52s 74ms/step - loss: 0.4589 - accuracy: 0.8522 - val_loss: 0.5288 - val_accuracy: 0.7837
Epoch 3/10
700/700 [==============================] - 52s 75ms/step - loss: 0.4042 - accuracy: 0.8707 - val_loss: 0.4997 - val_accuracy: 0.8065
Epoch 4/10
700/700 [==============================] - 52s 74ms/step - loss: 0.3797 - accuracy: 0.8808 - val_loss: 0.4528 - val_accuracy: 0.8127
Epoch 5/10
700/700 [==============================] - 53s 76ms/step - loss: 0.3645 - accuracy: 0.8853 - val_loss: 0.4799 - val_accuracy: 0.8372
Epoch 6/10
700/700 [==============================] - 52s 75ms/step - loss: 0.3578 - accuracy: 0.8907 - val_loss: 0.5499 - val_accuracy: 0.7995
Epoch 7/10
700/700 [==============================] - 52s 75ms/step - loss: 0.3577 - accuracy: 0.8919 - val_loss: 0.4129 - val_accura

In [41]:
time.sleep(400)

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 47s 62ms/step - loss: 0.7702 - accuracy: 0.7280 - val_loss: 0.5123 - val_accuracy: 0.8216
Epoch 2/10
744/744 [==============================] - 45s 61ms/step - loss: 0.4866 - accuracy: 0.8348 - val_loss: 0.4223 - val_accuracy: 0.8579
Epoch 3/10
744/744 [==============================] - 45s 61ms/step - loss: 0.4153 - accuracy: 0.8582 - val_loss: 0.3770 - val_accuracy: 0.8774
Epoch 4/10
744/744 [==============================] - 45s 61ms/step - loss: 0.3979 - accuracy: 0.8689 - val_loss: 0.3510 - val_accuracy: 0.8816
Epoch 5/10
744/744 [==============================] - 45s 61ms/step - loss: 0.3859 - accuracy: 0.8723 - val_loss: 0.3252 - val_accuracy: 0.8918
Epoch 6/10
744/744 [==============================] - 46s 61ms/step - loss: 0.3736 - accuracy: 0.8785 - val_loss: 0.3319 - val_accuracy: 0.8896
Epoch 7/10
744/744 [==============================] - 45s 61ms/step - loss: 0.3728 - accuracy: 0.8800 - val_loss: 0.3951 - val_accura

In [43]:
time.sleep(400)

In [44]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 48s 59ms/step - loss: 0.7781 - accuracy: 0.7306 - val_loss: 0.6109 - val_accuracy: 0.7729
Epoch 2/10
788/788 [==============================] - 47s 59ms/step - loss: 0.4847 - accuracy: 0.8376 - val_loss: 0.4372 - val_accuracy: 0.8623
Epoch 3/10
788/788 [==============================] - 47s 59ms/step - loss: 0.4238 - accuracy: 0.8599 - val_loss: 0.3844 - val_accuracy: 0.8722
Epoch 4/10
788/788 [==============================] - 47s 59ms/step - loss: 0.4052 - accuracy: 0.8687 - val_loss: 0.4283 - val_accuracy: 0.8631
Epoch 5/10
788/788 [==============================] - 47s 59ms/step - loss: 0.3989 - accuracy: 0.8696 - val_loss: 0.3270 - val_accuracy: 0.8882
Epoch 6/10
788/788 [==============================] - 47s 59ms/step - loss: 0.3949 - accuracy: 0.8730 - val_loss: 0.4917 - val_accuracy: 0.8529
Epoch 7/10
788/788 [==============================] - 47s 59ms/step - loss: 0.3940 - accuracy: 0.8726 - val_loss: 0.4197 - val_accura

In [45]:
time.sleep(400)

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 57s 67ms/step - loss: 0.7907 - accuracy: 0.7208 - val_loss: 0.3970 - val_accuracy: 0.8577
Epoch 2/10
832/832 [==============================] - 56s 67ms/step - loss: 0.4837 - accuracy: 0.8355 - val_loss: 0.3350 - val_accuracy: 0.8779
Epoch 3/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4242 - accuracy: 0.8609 - val_loss: 0.3354 - val_accuracy: 0.8791
Epoch 4/10
832/832 [==============================] - 56s 67ms/step - loss: 0.3995 - accuracy: 0.8689 - val_loss: 0.2987 - val_accuracy: 0.8920
Epoch 5/10
832/832 [==============================] - 56s 67ms/step - loss: 0.3830 - accuracy: 0.8762 - val_loss: 0.3267 - val_accuracy: 0.8946
Epoch 6/10
832/832 [==============================] - 56s 67ms/step - loss: 0.3799 - accuracy: 0.8788 - val_loss: 0.2848 - val_accuracy: 0.8995
Epoch 7/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3764 - accuracy: 0.8788 - val_loss: 0.2925 - val_accura

In [47]:
time.sleep(400)

In [48]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 60s 67ms/step - loss: 0.7241 - accuracy: 0.7477 - val_loss: 0.3741 - val_accuracy: 0.8689
Epoch 2/10
875/875 [==============================] - 59s 67ms/step - loss: 0.4549 - accuracy: 0.8468 - val_loss: 0.3127 - val_accuracy: 0.8865
Epoch 3/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3978 - accuracy: 0.8691 - val_loss: 0.2876 - val_accuracy: 0.8959
Epoch 4/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3767 - accuracy: 0.8772 - val_loss: 0.2917 - val_accuracy: 0.8973
Epoch 5/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3695 - accuracy: 0.8803 - val_loss: 0.2799 - val_accuracy: 0.9026
Epoch 6/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3642 - accuracy: 0.8848 - val_loss: 0.2872 - val_accuracy: 0.8976
Epoch 7/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3608 - accuracy: 0.8858 - val_loss: 0.2884 - val_accura

In [49]:
time.sleep(400)

In [50]:
len(models_lsa)

20

In [51]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [52]:
import gc

In [53]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [54]:
gc.collect()

45211

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [55]:
dsa_direction = "D:/guided-retraining/data/"+dataset+"/dsa_values.npy"

In [56]:
dsa_values = np.load(dsa_direction)

In [57]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)

In [58]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [59]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_dsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [60]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 150ms/step - loss: 1.8784 - accuracy: 0.3300 - val_loss: 1.4872 - val_accuracy: 0.4785
Epoch 2/10
44/44 [==============================] - 6s 148ms/step - loss: 1.4385 - accuracy: 0.4832 - val_loss: 1.3142 - val_accuracy: 0.5044
Epoch 3/10
44/44 [==============================] - 6s 147ms/step - loss: 1.2605 - accuracy: 0.5493 - val_loss: 1.0855 - val_accuracy: 0.5669
Epoch 4/10
44/44 [==============================] - 6s 149ms/step - loss: 1.1339 - accuracy: 0.5868 - val_loss: 0.9088 - val_accuracy: 0.6212
Epoch 5/10
44/44 [==============================] - 6s 146ms/step - loss: 1.0536 - accuracy: 0.6125 - val_loss: 0.7704 - val_accuracy: 0.6588
Epoch 6/10
44/44 [==============================] - 6s 148ms/step - loss: 0.9753 - accuracy: 0.6300 - val_loss: 0.7518 - val_accuracy: 0.6514
Epoch 7/10
44/44 [==============================] - 6s 147ms/step - loss: 0.9434 - accuracy: 0.6464 - val_loss: 0.7016 - val_accuracy: 0.6783
Epoc

In [61]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 10s 109ms/step - loss: 1.5299 - accuracy: 0.4404 - val_loss: 0.7328 - val_accuracy: 0.7407
Epoch 2/10
88/88 [==============================] - 9s 105ms/step - loss: 0.9777 - accuracy: 0.6459 - val_loss: 0.5871 - val_accuracy: 0.7754
Epoch 3/10
88/88 [==============================] - 9s 105ms/step - loss: 0.8387 - accuracy: 0.6961 - val_loss: 0.5521 - val_accuracy: 0.8063
Epoch 4/10
88/88 [==============================] - 9s 106ms/step - loss: 0.7550 - accuracy: 0.7307 - val_loss: 0.4770 - val_accuracy: 0.8206
Epoch 5/10
88/88 [==============================] - 9s 106ms/step - loss: 0.7114 - accuracy: 0.7411 - val_loss: 0.4691 - val_accuracy: 0.8244
Epoch 6/10
88/88 [==============================] - 9s 106ms/step - loss: 0.6583 - accuracy: 0.7650 - val_loss: 0.4464 - val_accuracy: 0.8340
Epoch 7/10
88/88 [==============================] - 9s 107ms/step - loss: 0.6052 - accuracy: 0.7807 - val_loss: 0.4361 - val_accuracy: 0.8388
Epo

In [62]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 13s 93ms/step - loss: 1.3076 - accuracy: 0.5254 - val_loss: 0.6146 - val_accuracy: 0.7606
Epoch 2/10
132/132 [==============================] - 12s 91ms/step - loss: 0.8000 - accuracy: 0.7092 - val_loss: 0.5389 - val_accuracy: 0.7938
Epoch 3/10
132/132 [==============================] - 12s 92ms/step - loss: 0.6708 - accuracy: 0.7568 - val_loss: 0.4914 - val_accuracy: 0.8076
Epoch 4/10
132/132 [==============================] - 12s 91ms/step - loss: 0.6016 - accuracy: 0.7876 - val_loss: 0.4566 - val_accuracy: 0.8243
Epoch 5/10
132/132 [==============================] - 12s 92ms/step - loss: 0.5573 - accuracy: 0.8002 - val_loss: 0.4228 - val_accuracy: 0.8421
Epoch 6/10
132/132 [==============================] - 12s 91ms/step - loss: 0.5221 - accuracy: 0.8171 - val_loss: 0.4103 - val_accuracy: 0.8560
Epoch 7/10
132/132 [==============================] - 12s 92ms/step - loss: 0.4880 - accuracy: 0.8304 - val_loss: 0.3970 - val_accurac

In [63]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 16s 84ms/step - loss: 1.1731 - accuracy: 0.5765 - val_loss: 0.5805 - val_accuracy: 0.7980
Epoch 2/10
175/175 [==============================] - 15s 83ms/step - loss: 0.7243 - accuracy: 0.7433 - val_loss: 0.4886 - val_accuracy: 0.8231
Epoch 3/10
175/175 [==============================] - 15s 83ms/step - loss: 0.6038 - accuracy: 0.7826 - val_loss: 0.4284 - val_accuracy: 0.8441
Epoch 4/10
175/175 [==============================] - 15s 83ms/step - loss: 0.5355 - accuracy: 0.8112 - val_loss: 0.4008 - val_accuracy: 0.8511
Epoch 5/10
175/175 [==============================] - 15s 83ms/step - loss: 0.4971 - accuracy: 0.8294 - val_loss: 0.3752 - val_accuracy: 0.8627
Epoch 6/10
175/175 [==============================] - 15s 84ms/step - loss: 0.4583 - accuracy: 0.8413 - val_loss: 0.3711 - val_accuracy: 0.8658
Epoch 7/10
175/175 [==============================] - 15s 84ms/step - loss: 0.4258 - accuracy: 0.8479 - val_loss: 0.3634 - val_accurac

In [64]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 19s 82ms/step - loss: 1.0917 - accuracy: 0.6026 - val_loss: 0.5447 - val_accuracy: 0.7907
Epoch 2/10
219/219 [==============================] - 18s 82ms/step - loss: 0.6685 - accuracy: 0.7623 - val_loss: 0.4575 - val_accuracy: 0.8361
Epoch 3/10
219/219 [==============================] - 18s 82ms/step - loss: 0.5697 - accuracy: 0.8002 - val_loss: 0.4029 - val_accuracy: 0.8497
Epoch 4/10
219/219 [==============================] - 18s 82ms/step - loss: 0.5076 - accuracy: 0.8239 - val_loss: 0.3893 - val_accuracy: 0.8517
Epoch 5/10
219/219 [==============================] - 18s 82ms/step - loss: 0.4704 - accuracy: 0.8401 - val_loss: 0.3552 - val_accuracy: 0.8678
Epoch 6/10
219/219 [==============================] - 18s 82ms/step - loss: 0.4316 - accuracy: 0.8494 - val_loss: 0.3583 - val_accuracy: 0.8711
Epoch 7/10
219/219 [==============================] - 18s 82ms/step - loss: 0.4124 - accuracy: 0.8601 - val_loss: 0.3403 - val_accurac

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 22s 81ms/step - loss: 1.0501 - accuracy: 0.6218 - val_loss: 0.5188 - val_accuracy: 0.8066
Epoch 2/10
263/263 [==============================] - 21s 81ms/step - loss: 0.6317 - accuracy: 0.7814 - val_loss: 0.4387 - val_accuracy: 0.8424
Epoch 3/10
263/263 [==============================] - 21s 82ms/step - loss: 0.5399 - accuracy: 0.8171 - val_loss: 0.3804 - val_accuracy: 0.8628
Epoch 4/10
263/263 [==============================] - 22s 82ms/step - loss: 0.4786 - accuracy: 0.8381 - val_loss: 0.3866 - val_accuracy: 0.8620
Epoch 5/10
263/263 [==============================] - 22s 83ms/step - loss: 0.4401 - accuracy: 0.8529 - val_loss: 0.3573 - val_accuracy: 0.8739
Epoch 6/10
263/263 [==============================] - 22s 84ms/step - loss: 0.4127 - accuracy: 0.8620 - val_loss: 0.3375 - val_accuracy: 0.8770
Epoch 7/10
263/263 [==============================] - 22s 84ms/step - loss: 0.3898 - accuracy: 0.8714 - val_loss: 0.3150 - val_accurac

In [66]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 27s 84ms/step - loss: 1.0179 - accuracy: 0.6356 - val_loss: 0.5479 - val_accuracy: 0.7861
Epoch 2/10
307/307 [==============================] - 26s 85ms/step - loss: 0.6187 - accuracy: 0.7869 - val_loss: 0.4399 - val_accuracy: 0.8376
Epoch 3/10
307/307 [==============================] - 26s 84ms/step - loss: 0.5243 - accuracy: 0.8214 - val_loss: 0.3839 - val_accuracy: 0.8596
Epoch 4/10
307/307 [==============================] - 26s 84ms/step - loss: 0.4713 - accuracy: 0.8423 - val_loss: 0.3730 - val_accuracy: 0.8677
Epoch 5/10
307/307 [==============================] - 26s 84ms/step - loss: 0.4242 - accuracy: 0.8574 - val_loss: 0.3329 - val_accuracy: 0.8766
Epoch 6/10
307/307 [==============================] - 26s 84ms/step - loss: 0.4073 - accuracy: 0.8615 - val_loss: 0.3369 - val_accuracy: 0.8796
Epoch 7/10
307/307 [==============================] - 26s 84ms/step - loss: 0.3855 - accuracy: 0.8712 - val_loss: 0.3559 - val_accurac

In [67]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 30s 82ms/step - loss: 0.9336 - accuracy: 0.6625 - val_loss: 0.4840 - val_accuracy: 0.8157
Epoch 2/10
350/350 [==============================] - 28s 81ms/step - loss: 0.5690 - accuracy: 0.8040 - val_loss: 0.4137 - val_accuracy: 0.8471
Epoch 3/10
350/350 [==============================] - 28s 81ms/step - loss: 0.4900 - accuracy: 0.8339 - val_loss: 0.3559 - val_accuracy: 0.8691
Epoch 4/10
350/350 [==============================] - 29s 83ms/step - loss: 0.4469 - accuracy: 0.8506 - val_loss: 0.3458 - val_accuracy: 0.8731
Epoch 5/10
350/350 [==============================] - 29s 82ms/step - loss: 0.4114 - accuracy: 0.8607 - val_loss: 0.3219 - val_accuracy: 0.8887
Epoch 6/10
350/350 [==============================] - 28s 81ms/step - loss: 0.3745 - accuracy: 0.8741 - val_loss: 0.3226 - val_accuracy: 0.8832
Epoch 7/10
350/350 [==============================] - 29s 82ms/step - loss: 0.3665 - accuracy: 0.8785 - val_loss: 0.3171 - val_accurac

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 33s 81ms/step - loss: 0.9485 - accuracy: 0.6623 - val_loss: 0.4757 - val_accuracy: 0.8261
Epoch 2/10
394/394 [==============================] - 32s 81ms/step - loss: 0.5494 - accuracy: 0.8068 - val_loss: 0.3898 - val_accuracy: 0.8573
Epoch 3/10
394/394 [==============================] - 32s 81ms/step - loss: 0.4677 - accuracy: 0.8424 - val_loss: 0.3588 - val_accuracy: 0.8695
Epoch 4/10
394/394 [==============================] - 32s 82ms/step - loss: 0.4304 - accuracy: 0.8563 - val_loss: 0.3453 - val_accuracy: 0.8786
Epoch 5/10
394/394 [==============================] - 32s 81ms/step - loss: 0.3989 - accuracy: 0.8676 - val_loss: 0.3172 - val_accuracy: 0.8834
Epoch 6/10
394/394 [==============================] - 32s 81ms/step - loss: 0.3779 - accuracy: 0.8720 - val_loss: 0.3187 - val_accuracy: 0.8879
Epoch 7/10
394/394 [==============================] - 32s 82ms/step - loss: 0.3581 - accuracy: 0.8799 - val_loss: 0.3264 - val_accurac

In [69]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 36s 81ms/step - loss: 0.8969 - accuracy: 0.6824 - val_loss: 0.4460 - val_accuracy: 0.8390
Epoch 2/10
438/438 [==============================] - 35s 80ms/step - loss: 0.5238 - accuracy: 0.8177 - val_loss: 0.3729 - val_accuracy: 0.8632
Epoch 3/10
438/438 [==============================] - 35s 81ms/step - loss: 0.4586 - accuracy: 0.8468 - val_loss: 0.3449 - val_accuracy: 0.8731
Epoch 4/10
438/438 [==============================] - 36s 82ms/step - loss: 0.4167 - accuracy: 0.8592 - val_loss: 0.3213 - val_accuracy: 0.8814
Epoch 5/10
438/438 [==============================] - 35s 80ms/step - loss: 0.3854 - accuracy: 0.8691 - val_loss: 0.3068 - val_accuracy: 0.8884
Epoch 6/10
438/438 [==============================] - 35s 81ms/step - loss: 0.3665 - accuracy: 0.8776 - val_loss: 0.3085 - val_accuracy: 0.8869
Epoch 7/10
438/438 [==============================] - 35s 81ms/step - loss: 0.3534 - accuracy: 0.8833 - val_loss: 0.2966 - val_accurac

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 40s 81ms/step - loss: 0.8814 - accuracy: 0.6814 - val_loss: 0.4550 - val_accuracy: 0.8274
Epoch 2/10
482/482 [==============================] - 39s 81ms/step - loss: 0.5298 - accuracy: 0.8186 - val_loss: 0.3823 - val_accuracy: 0.8616
Epoch 3/10
482/482 [==============================] - 39s 80ms/step - loss: 0.4570 - accuracy: 0.8451 - val_loss: 0.3662 - val_accuracy: 0.8695
Epoch 4/10
482/482 [==============================] - 39s 81ms/step - loss: 0.4105 - accuracy: 0.8626 - val_loss: 0.3310 - val_accuracy: 0.8817
Epoch 5/10
482/482 [==============================] - 39s 81ms/step - loss: 0.3868 - accuracy: 0.8716 - val_loss: 0.2987 - val_accuracy: 0.8920
Epoch 6/10
482/482 [==============================] - 39s 81ms/step - loss: 0.3703 - accuracy: 0.8787 - val_loss: 0.3238 - val_accuracy: 0.8874
Epoch 7/10
482/482 [==============================] - 39s 80ms/step - loss: 0.3547 - accuracy: 0.8820 - val_loss: 0.2940 - val_accura

In [71]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 42s 78ms/step - loss: 0.8411 - accuracy: 0.6963 - val_loss: 0.4385 - val_accuracy: 0.8453
Epoch 2/10
525/525 [==============================] - 41s 78ms/step - loss: 0.5161 - accuracy: 0.8221 - val_loss: 0.3582 - val_accuracy: 0.8709
Epoch 3/10
525/525 [==============================] - 41s 78ms/step - loss: 0.4439 - accuracy: 0.8514 - val_loss: 0.3404 - val_accuracy: 0.8724
Epoch 4/10
525/525 [==============================] - 41s 78ms/step - loss: 0.4061 - accuracy: 0.8666 - val_loss: 0.3172 - val_accuracy: 0.8840
Epoch 5/10
525/525 [==============================] - 41s 78ms/step - loss: 0.3776 - accuracy: 0.8757 - val_loss: 0.2971 - val_accuracy: 0.8956
Epoch 6/10
525/525 [==============================] - 41s 78ms/step - loss: 0.3642 - accuracy: 0.8791 - val_loss: 0.3083 - val_accuracy: 0.8919
Epoch 7/10
525/525 [==============================] - 41s 78ms/step - loss: 0.3479 - accuracy: 0.8847 - val_loss: 0.3080 - val_accura

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 46s 80ms/step - loss: 0.8073 - accuracy: 0.7131 - val_loss: 0.4310 - val_accuracy: 0.8508
Epoch 2/10
569/569 [==============================] - 45s 79ms/step - loss: 0.4905 - accuracy: 0.8350 - val_loss: 0.3443 - val_accuracy: 0.8779
Epoch 3/10
569/569 [==============================] - 45s 79ms/step - loss: 0.4253 - accuracy: 0.8587 - val_loss: 0.3218 - val_accuracy: 0.8829
Epoch 4/10
569/569 [==============================] - 45s 80ms/step - loss: 0.3867 - accuracy: 0.8728 - val_loss: 0.3068 - val_accuracy: 0.8869
Epoch 5/10
569/569 [==============================] - 45s 79ms/step - loss: 0.3650 - accuracy: 0.8794 - val_loss: 0.2999 - val_accuracy: 0.8928
Epoch 6/10
569/569 [==============================] - 45s 79ms/step - loss: 0.3564 - accuracy: 0.8821 - val_loss: 0.3038 - val_accuracy: 0.8991
Epoch 7/10
569/569 [==============================] - 45s 79ms/step - loss: 0.3476 - accuracy: 0.8873 - val_loss: 0.2909 - val_accura

In [73]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 50s 80ms/step - loss: 0.8027 - accuracy: 0.7158 - val_loss: 0.4279 - val_accuracy: 0.8479
Epoch 2/10
613/613 [==============================] - 48s 79ms/step - loss: 0.4914 - accuracy: 0.8321 - val_loss: 0.3725 - val_accuracy: 0.8651
Epoch 3/10
613/613 [==============================] - 48s 79ms/step - loss: 0.4258 - accuracy: 0.8579 - val_loss: 0.3155 - val_accuracy: 0.8832
Epoch 4/10
613/613 [==============================] - 48s 78ms/step - loss: 0.3858 - accuracy: 0.8692 - val_loss: 0.2963 - val_accuracy: 0.8915
Epoch 5/10
613/613 [==============================] - 48s 78ms/step - loss: 0.3665 - accuracy: 0.8791 - val_loss: 0.2948 - val_accuracy: 0.8934
Epoch 6/10
613/613 [==============================] - 48s 78ms/step - loss: 0.3505 - accuracy: 0.8838 - val_loss: 0.3319 - val_accuracy: 0.9012
Epoch 7/10
613/613 [==============================] - 48s 78ms/step - loss: 0.3442 - accuracy: 0.8888 - val_loss: 0.3079 - val_accura

In [74]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 52s 78ms/step - loss: 0.8041 - accuracy: 0.7220 - val_loss: 0.4016 - val_accuracy: 0.8553
Epoch 2/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4774 - accuracy: 0.8402 - val_loss: 0.3345 - val_accuracy: 0.8768
Epoch 3/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4151 - accuracy: 0.8630 - val_loss: 0.3094 - val_accuracy: 0.8885
Epoch 4/10
657/657 [==============================] - 51s 78ms/step - loss: 0.3844 - accuracy: 0.8712 - val_loss: 0.3158 - val_accuracy: 0.8835
Epoch 5/10
657/657 [==============================] - 51s 78ms/step - loss: 0.3694 - accuracy: 0.8779 - val_loss: 0.3366 - val_accuracy: 0.8890
Epoch 6/10
657/657 [==============================] - 51s 77ms/step - loss: 0.3567 - accuracy: 0.8825 - val_loss: 0.3486 - val_accuracy: 0.8914
Epoch 7/10
657/657 [==============================] - 51s 77ms/step - loss: 0.3566 - accuracy: 0.8853 - val_loss: 0.3724 - val_accura

In [75]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 54s 76ms/step - loss: 0.7705 - accuracy: 0.7295 - val_loss: 0.4028 - val_accuracy: 0.8534
Epoch 2/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4725 - accuracy: 0.8393 - val_loss: 0.3188 - val_accuracy: 0.8872
Epoch 3/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4120 - accuracy: 0.8614 - val_loss: 0.2968 - val_accuracy: 0.8939
Epoch 4/10
700/700 [==============================] - 53s 76ms/step - loss: 0.3813 - accuracy: 0.8719 - val_loss: 0.3006 - val_accuracy: 0.8934
Epoch 5/10
700/700 [==============================] - 53s 76ms/step - loss: 0.3756 - accuracy: 0.8776 - val_loss: 0.2935 - val_accuracy: 0.8944
Epoch 6/10
700/700 [==============================] - 53s 76ms/step - loss: 0.3700 - accuracy: 0.8797 - val_loss: 0.2862 - val_accuracy: 0.9001
Epoch 7/10
700/700 [==============================] - 53s 75ms/step - loss: 0.3625 - accuracy: 0.8821 - val_loss: 0.3146 - val_accura

In [76]:
time.sleep(400)

In [77]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 52s 69ms/step - loss: 0.7781 - accuracy: 0.7224 - val_loss: 0.4053 - val_accuracy: 0.8581
Epoch 2/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4763 - accuracy: 0.8392 - val_loss: 0.3386 - val_accuracy: 0.8744
Epoch 3/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4158 - accuracy: 0.8629 - val_loss: 0.3293 - val_accuracy: 0.8801
Epoch 4/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3857 - accuracy: 0.8723 - val_loss: 0.3044 - val_accuracy: 0.8890
Epoch 5/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3688 - accuracy: 0.8809 - val_loss: 0.3035 - val_accuracy: 0.8937
Epoch 6/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3607 - accuracy: 0.8812 - val_loss: 0.2812 - val_accuracy: 0.9000
Epoch 7/10
744/744 [==============================] - 52s 70ms/step - loss: 0.3629 - accuracy: 0.8842 - val_loss: 0.3071 - val_accura

In [78]:
time.sleep(400)

In [79]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 54s 68ms/step - loss: 0.7343 - accuracy: 0.7431 - val_loss: 0.3818 - val_accuracy: 0.8589
Epoch 2/10
788/788 [==============================] - 54s 68ms/step - loss: 0.4478 - accuracy: 0.8483 - val_loss: 0.3385 - val_accuracy: 0.8784
Epoch 3/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3944 - accuracy: 0.8688 - val_loss: 0.2869 - val_accuracy: 0.8955
Epoch 4/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3723 - accuracy: 0.8765 - val_loss: 0.3240 - val_accuracy: 0.8889
Epoch 5/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3603 - accuracy: 0.8841 - val_loss: 0.3148 - val_accuracy: 0.8866
Epoch 6/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3626 - accuracy: 0.8826 - val_loss: 0.3364 - val_accuracy: 0.8928
Epoch 7/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3633 - accuracy: 0.8837 - val_loss: 0.2835 - val_accura

In [80]:
time.sleep(400)

In [81]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 58s 69ms/step - loss: 0.7237 - accuracy: 0.7463 - val_loss: 0.3949 - val_accuracy: 0.8562
Epoch 2/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4501 - accuracy: 0.8495 - val_loss: 0.3163 - val_accuracy: 0.8833
Epoch 3/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3967 - accuracy: 0.8685 - val_loss: 0.3085 - val_accuracy: 0.8859
Epoch 4/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3743 - accuracy: 0.8767 - val_loss: 0.3165 - val_accuracy: 0.8891
Epoch 5/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3701 - accuracy: 0.8788 - val_loss: 0.3183 - val_accuracy: 0.8892
Epoch 6/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3612 - accuracy: 0.8843 - val_loss: 0.3481 - val_accuracy: 0.8773
Epoch 7/10
832/832 [==============================] - 57s 69ms/step - loss: 0.3665 - accuracy: 0.8847 - val_loss: 0.3129 - val_accura

In [82]:
time.sleep(400)

In [83]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 60s 68ms/step - loss: 0.7129 - accuracy: 0.7500 - val_loss: 0.3847 - val_accuracy: 0.8600
Epoch 2/10
875/875 [==============================] - 59s 68ms/step - loss: 0.4444 - accuracy: 0.8520 - val_loss: 0.3049 - val_accuracy: 0.8889
Epoch 3/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3913 - accuracy: 0.8715 - val_loss: 0.2854 - val_accuracy: 0.8966
Epoch 4/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3707 - accuracy: 0.8782 - val_loss: 0.3077 - val_accuracy: 0.8943
Epoch 5/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3655 - accuracy: 0.8812 - val_loss: 0.3071 - val_accuracy: 0.8977
Epoch 6/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3606 - accuracy: 0.8840 - val_loss: 0.2781 - val_accuracy: 0.9015
Epoch 7/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3596 - accuracy: 0.8852 - val_loss: 0.2865 - val_accura

In [84]:
time.sleep(400)

In [85]:
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [86]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [87]:
gc.collect()

89881

## Uncertainty

In [88]:
dg_direction = "D:/guided-retraining/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [89]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [90]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [91]:
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_dg.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [92]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 150ms/step - loss: 1.9706 - accuracy: 0.2679 - val_loss: 2.1915 - val_accuracy: 0.1504
Epoch 2/10
44/44 [==============================] - 6s 145ms/step - loss: 1.7569 - accuracy: 0.2979 - val_loss: 2.1105 - val_accuracy: 0.2756
Epoch 3/10
44/44 [==============================] - 6s 143ms/step - loss: 1.6679 - accuracy: 0.3146 - val_loss: 1.9619 - val_accuracy: 0.3084
Epoch 4/10
44/44 [==============================] - 6s 144ms/step - loss: 1.5877 - accuracy: 0.3593 - val_loss: 1.8529 - val_accuracy: 0.4211
Epoch 5/10
44/44 [==============================] - 6s 144ms/step - loss: 1.5394 - accuracy: 0.3507 - val_loss: 1.8332 - val_accuracy: 0.4354
Epoch 6/10
44/44 [==============================] - 6s 149ms/step - loss: 1.4941 - accuracy: 0.3661 - val_loss: 1.7351 - val_accuracy: 0.4235
Epoch 7/10
44/44 [==============================] - 6s 145ms/step - loss: 1.4524 - accuracy: 0.3839 - val_loss: 1.6727 - val_accuracy: 0.4900
Epoc

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 10s 108ms/step - loss: 1.8208 - accuracy: 0.2868 - val_loss: 2.0210 - val_accuracy: 0.2436
Epoch 2/10
88/88 [==============================] - 9s 104ms/step - loss: 1.4968 - accuracy: 0.3866 - val_loss: 1.7558 - val_accuracy: 0.3798
Epoch 3/10
88/88 [==============================] - 9s 104ms/step - loss: 1.3720 - accuracy: 0.4239 - val_loss: 1.8373 - val_accuracy: 0.3737
Epoch 4/10
88/88 [==============================] - 9s 106ms/step - loss: 1.3123 - accuracy: 0.4275 - val_loss: 1.5807 - val_accuracy: 0.3987
Epoch 5/10
88/88 [==============================] - 9s 104ms/step - loss: 1.2804 - accuracy: 0.4457 - val_loss: 1.5997 - val_accuracy: 0.4126
Epoch 6/10
88/88 [==============================] - 9s 104ms/step - loss: 1.2194 - accuracy: 0.4643 - val_loss: 1.6400 - val_accuracy: 0.4572
Epoch 7/10
88/88 [==============================] - 9s 105ms/step - loss: 1.1860 - accuracy: 0.4796 - val_loss: 1.6231 - val_accuracy: 0.4226
Epo

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 13s 92ms/step - loss: 1.6286 - accuracy: 0.3500 - val_loss: 1.7689 - val_accuracy: 0.4355
Epoch 2/10
132/132 [==============================] - 12s 91ms/step - loss: 1.3361 - accuracy: 0.4386 - val_loss: 1.6020 - val_accuracy: 0.4177
Epoch 3/10
132/132 [==============================] - 12s 91ms/step - loss: 1.2458 - accuracy: 0.4626 - val_loss: 1.6521 - val_accuracy: 0.4664
Epoch 4/10
132/132 [==============================] - 12s 89ms/step - loss: 1.1885 - accuracy: 0.4888 - val_loss: 1.5523 - val_accuracy: 0.4948
Epoch 5/10
132/132 [==============================] - 12s 89ms/step - loss: 1.1500 - accuracy: 0.4910 - val_loss: 1.4033 - val_accuracy: 0.4458
Epoch 6/10
132/132 [==============================] - 12s 91ms/step - loss: 1.1206 - accuracy: 0.5095 - val_loss: 1.4932 - val_accuracy: 0.4194
Epoch 7/10
132/132 [==============================] - 12s 91ms/step - loss: 1.0913 - accuracy: 0.5126 - val_loss: 1.3034 - val_accurac

In [95]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 16s 86ms/step - loss: 1.5738 - accuracy: 0.3736 - val_loss: 1.6834 - val_accuracy: 0.4335
Epoch 2/10
175/175 [==============================] - 15s 83ms/step - loss: 1.2711 - accuracy: 0.4589 - val_loss: 1.5633 - val_accuracy: 0.4270
Epoch 3/10
175/175 [==============================] - 15s 84ms/step - loss: 1.1797 - accuracy: 0.4877 - val_loss: 1.5308 - val_accuracy: 0.4256
Epoch 4/10
175/175 [==============================] - 15s 83ms/step - loss: 1.1230 - accuracy: 0.5067 - val_loss: 1.3280 - val_accuracy: 0.5111
Epoch 5/10
175/175 [==============================] - 15s 86ms/step - loss: 1.0748 - accuracy: 0.5318 - val_loss: 1.4011 - val_accuracy: 0.5112
Epoch 6/10
175/175 [==============================] - 15s 87ms/step - loss: 1.0460 - accuracy: 0.5440 - val_loss: 1.3409 - val_accuracy: 0.5158
Epoch 7/10
175/175 [==============================] - 15s 85ms/step - loss: 1.0156 - accuracy: 0.5641 - val_loss: 1.2526 - val_accurac

In [96]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 19s 82ms/step - loss: 1.4622 - accuracy: 0.4009 - val_loss: 1.6924 - val_accuracy: 0.4534
Epoch 2/10
219/219 [==============================] - 18s 82ms/step - loss: 1.1870 - accuracy: 0.4891 - val_loss: 1.5882 - val_accuracy: 0.5002
Epoch 3/10
219/219 [==============================] - 18s 82ms/step - loss: 1.1049 - accuracy: 0.5191 - val_loss: 1.4046 - val_accuracy: 0.4841
Epoch 4/10
219/219 [==============================] - 18s 82ms/step - loss: 1.0487 - accuracy: 0.5454 - val_loss: 1.2870 - val_accuracy: 0.5281
Epoch 5/10
219/219 [==============================] - 18s 82ms/step - loss: 1.0095 - accuracy: 0.5711 - val_loss: 1.2257 - val_accuracy: 0.5581
Epoch 6/10
219/219 [==============================] - 18s 83ms/step - loss: 0.9650 - accuracy: 0.5839 - val_loss: 1.2661 - val_accuracy: 0.5586
Epoch 7/10
219/219 [==============================] - 18s 83ms/step - loss: 0.9351 - accuracy: 0.6047 - val_loss: 1.2092 - val_accurac

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 22s 82ms/step - loss: 1.4424 - accuracy: 0.4168 - val_loss: 1.5112 - val_accuracy: 0.5139
Epoch 2/10
263/263 [==============================] - 21s 81ms/step - loss: 1.1397 - accuracy: 0.5140 - val_loss: 1.4163 - val_accuracy: 0.5580
Epoch 3/10
263/263 [==============================] - 21s 81ms/step - loss: 1.0337 - accuracy: 0.5724 - val_loss: 1.3383 - val_accuracy: 0.5651
Epoch 4/10
263/263 [==============================] - 22s 82ms/step - loss: 0.9717 - accuracy: 0.6034 - val_loss: 1.1922 - val_accuracy: 0.5906
Epoch 5/10
263/263 [==============================] - 22s 83ms/step - loss: 0.9251 - accuracy: 0.6267 - val_loss: 1.1639 - val_accuracy: 0.5942
Epoch 6/10
263/263 [==============================] - 22s 82ms/step - loss: 0.8852 - accuracy: 0.6451 - val_loss: 1.2197 - val_accuracy: 0.5951
Epoch 7/10
263/263 [==============================] - 22s 84ms/step - loss: 0.8512 - accuracy: 0.6676 - val_loss: 1.1035 - val_accurac

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 27s 84ms/step - loss: 1.3613 - accuracy: 0.4577 - val_loss: 1.5744 - val_accuracy: 0.5191
Epoch 2/10
307/307 [==============================] - 26s 84ms/step - loss: 1.0577 - accuracy: 0.5741 - val_loss: 1.2175 - val_accuracy: 0.5693
Epoch 3/10
307/307 [==============================] - 26s 84ms/step - loss: 0.9522 - accuracy: 0.6235 - val_loss: 1.1525 - val_accuracy: 0.5714
Epoch 4/10
307/307 [==============================] - 27s 87ms/step - loss: 0.8925 - accuracy: 0.6563 - val_loss: 1.1410 - val_accuracy: 0.6041
Epoch 5/10
307/307 [==============================] - 26s 84ms/step - loss: 0.8336 - accuracy: 0.6860 - val_loss: 1.0595 - val_accuracy: 0.6250
Epoch 6/10
307/307 [==============================] - 26s 83ms/step - loss: 0.7869 - accuracy: 0.7056 - val_loss: 1.0331 - val_accuracy: 0.6081
Epoch 7/10
307/307 [==============================] - 26s 84ms/step - loss: 0.7544 - accuracy: 0.7198 - val_loss: 0.8972 - val_accurac

In [99]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 30s 82ms/step - loss: 1.3163 - accuracy: 0.4874 - val_loss: 1.3506 - val_accuracy: 0.5594
Epoch 2/10
350/350 [==============================] - 29s 82ms/step - loss: 0.9971 - accuracy: 0.6144 - val_loss: 1.1735 - val_accuracy: 0.5792
Epoch 3/10
350/350 [==============================] - 28s 81ms/step - loss: 0.8829 - accuracy: 0.6684 - val_loss: 1.0759 - val_accuracy: 0.5982
Epoch 4/10
350/350 [==============================] - 28s 81ms/step - loss: 0.8121 - accuracy: 0.7027 - val_loss: 1.0312 - val_accuracy: 0.6126
Epoch 5/10
350/350 [==============================] - 28s 81ms/step - loss: 0.7610 - accuracy: 0.7272 - val_loss: 0.9823 - val_accuracy: 0.6144
Epoch 6/10
350/350 [==============================] - 29s 82ms/step - loss: 0.7249 - accuracy: 0.7425 - val_loss: 0.9185 - val_accuracy: 0.6567
Epoch 7/10
350/350 [==============================] - 29s 83ms/step - loss: 0.7036 - accuracy: 0.7496 - val_loss: 0.9548 - val_accurac

In [100]:
n=8
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 33s 82ms/step - loss: 1.2402 - accuracy: 0.5244 - val_loss: 1.2932 - val_accuracy: 0.5609
Epoch 2/10
394/394 [==============================] - 33s 84ms/step - loss: 0.9122 - accuracy: 0.6605 - val_loss: 1.1653 - val_accuracy: 0.5983
Epoch 3/10
394/394 [==============================] - 32s 81ms/step - loss: 0.8054 - accuracy: 0.7104 - val_loss: 0.9820 - val_accuracy: 0.6171
Epoch 4/10
394/394 [==============================] - 32s 81ms/step - loss: 0.7448 - accuracy: 0.7349 - val_loss: 0.9542 - val_accuracy: 0.6371
Epoch 5/10
394/394 [==============================] - 32s 80ms/step - loss: 0.6966 - accuracy: 0.7538 - val_loss: 0.7515 - val_accuracy: 0.6844
Epoch 6/10
394/394 [==============================] - 32s 80ms/step - loss: 0.6662 - accuracy: 0.7680 - val_loss: 0.8501 - val_accuracy: 0.7130
Epoch 7/10
394/394 [==============================] - 32s 80ms/step - loss: 0.6445 - accuracy: 0.7762 - val_loss: 0.7804 - val_accurac

In [101]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 36s 81ms/step - loss: 1.1873 - accuracy: 0.5509 - val_loss: 1.2345 - val_accuracy: 0.5701
Epoch 2/10
438/438 [==============================] - 35s 81ms/step - loss: 0.8459 - accuracy: 0.6983 - val_loss: 1.0784 - val_accuracy: 0.5856
Epoch 3/10
438/438 [==============================] - 35s 80ms/step - loss: 0.7447 - accuracy: 0.7393 - val_loss: 0.8290 - val_accuracy: 0.6472
Epoch 4/10
438/438 [==============================] - 35s 80ms/step - loss: 0.6889 - accuracy: 0.7621 - val_loss: 0.9023 - val_accuracy: 0.6556
Epoch 5/10
438/438 [==============================] - 35s 80ms/step - loss: 0.6489 - accuracy: 0.7782 - val_loss: 0.7282 - val_accuracy: 0.6981
Epoch 6/10
438/438 [==============================] - 35s 79ms/step - loss: 0.6212 - accuracy: 0.7878 - val_loss: 0.7462 - val_accuracy: 0.7206
Epoch 7/10
438/438 [==============================] - 35s 79ms/step - loss: 0.6030 - accuracy: 0.7947 - val_loss: 0.7318 - val_accurac

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 39s 80ms/step - loss: 1.1244 - accuracy: 0.5914 - val_loss: 1.1982 - val_accuracy: 0.5572
Epoch 2/10
482/482 [==============================] - 38s 80ms/step - loss: 0.7879 - accuracy: 0.7275 - val_loss: 1.0164 - val_accuracy: 0.6079
Epoch 3/10
482/482 [==============================] - 39s 81ms/step - loss: 0.6933 - accuracy: 0.7620 - val_loss: 0.8735 - val_accuracy: 0.6436
Epoch 4/10
482/482 [==============================] - 39s 80ms/step - loss: 0.6444 - accuracy: 0.7795 - val_loss: 0.9730 - val_accuracy: 0.6121
Epoch 5/10
482/482 [==============================] - 38s 80ms/step - loss: 0.6121 - accuracy: 0.7915 - val_loss: 0.8089 - val_accuracy: 0.6770
Epoch 6/10
482/482 [==============================] - 38s 79ms/step - loss: 0.5839 - accuracy: 0.8020 - val_loss: 0.8048 - val_accuracy: 0.6800
Epoch 7/10
482/482 [==============================] - 38s 80ms/step - loss: 0.5719 - accuracy: 0.8086 - val_loss: 0.7359 - val_accura

In [103]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 42s 78ms/step - loss: 1.0458 - accuracy: 0.6304 - val_loss: 0.9834 - val_accuracy: 0.5939
Epoch 2/10
525/525 [==============================] - 40s 77ms/step - loss: 0.7175 - accuracy: 0.7546 - val_loss: 0.7498 - val_accuracy: 0.6982
Epoch 3/10
525/525 [==============================] - 40s 77ms/step - loss: 0.6420 - accuracy: 0.7818 - val_loss: 0.6447 - val_accuracy: 0.7551
Epoch 4/10
525/525 [==============================] - 40s 77ms/step - loss: 0.5979 - accuracy: 0.7958 - val_loss: 0.6078 - val_accuracy: 0.7647
Epoch 5/10
525/525 [==============================] - 40s 76ms/step - loss: 0.5721 - accuracy: 0.8075 - val_loss: 0.5025 - val_accuracy: 0.8086
Epoch 6/10
525/525 [==============================] - 40s 76ms/step - loss: 0.5533 - accuracy: 0.8143 - val_loss: 0.4940 - val_accuracy: 0.8194
Epoch 7/10
525/525 [==============================] - 40s 76ms/step - loss: 0.5353 - accuracy: 0.8211 - val_loss: 0.4325 - val_accura

In [104]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 45s 78ms/step - loss: 1.0097 - accuracy: 0.6418 - val_loss: 0.8294 - val_accuracy: 0.6342
Epoch 2/10
569/569 [==============================] - 44s 77ms/step - loss: 0.6834 - accuracy: 0.7630 - val_loss: 0.5801 - val_accuracy: 0.7850
Epoch 3/10
569/569 [==============================] - 46s 81ms/step - loss: 0.6001 - accuracy: 0.7925 - val_loss: 0.4348 - val_accuracy: 0.8534
Epoch 4/10
569/569 [==============================] - 45s 78ms/step - loss: 0.5530 - accuracy: 0.8122 - val_loss: 0.3669 - val_accuracy: 0.8736
Epoch 5/10
569/569 [==============================] - 45s 79ms/step - loss: 0.5353 - accuracy: 0.8226 - val_loss: 0.3388 - val_accuracy: 0.8842
Epoch 6/10
569/569 [==============================] - 45s 78ms/step - loss: 0.5119 - accuracy: 0.8307 - val_loss: 0.3174 - val_accuracy: 0.8880
Epoch 7/10
569/569 [==============================] - 45s 79ms/step - loss: 0.4974 - accuracy: 0.8368 - val_loss: 0.2927 - val_accura

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 48s 77ms/step - loss: 0.9662 - accuracy: 0.6566 - val_loss: 0.5486 - val_accuracy: 0.8099
Epoch 2/10
613/613 [==============================] - 47s 77ms/step - loss: 0.6499 - accuracy: 0.7777 - val_loss: 0.4094 - val_accuracy: 0.8526
Epoch 3/10
613/613 [==============================] - 47s 76ms/step - loss: 0.5625 - accuracy: 0.8091 - val_loss: 0.3977 - val_accuracy: 0.8679
Epoch 4/10
613/613 [==============================] - 47s 77ms/step - loss: 0.5163 - accuracy: 0.8263 - val_loss: 0.3030 - val_accuracy: 0.8924
Epoch 5/10
613/613 [==============================] - 47s 76ms/step - loss: 0.4983 - accuracy: 0.8328 - val_loss: 0.2921 - val_accuracy: 0.8982
Epoch 6/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4819 - accuracy: 0.8392 - val_loss: 0.2839 - val_accuracy: 0.8993
Epoch 7/10
613/613 [==============================] - 47s 76ms/step - loss: 0.4684 - accuracy: 0.8458 - val_loss: 0.3335 - val_accura

In [106]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 53s 79ms/step - loss: 0.9419 - accuracy: 0.6701 - val_loss: 0.4533 - val_accuracy: 0.8419
Epoch 2/10
657/657 [==============================] - 51s 78ms/step - loss: 0.6099 - accuracy: 0.7926 - val_loss: 0.3695 - val_accuracy: 0.8711
Epoch 3/10
657/657 [==============================] - 52s 78ms/step - loss: 0.5253 - accuracy: 0.8231 - val_loss: 0.3124 - val_accuracy: 0.8931
Epoch 4/10
657/657 [==============================] - 52s 78ms/step - loss: 0.4848 - accuracy: 0.8401 - val_loss: 0.2910 - val_accuracy: 0.8962
Epoch 5/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4616 - accuracy: 0.8483 - val_loss: 0.2698 - val_accuracy: 0.9036
Epoch 6/10
657/657 [==============================] - 52s 79ms/step - loss: 0.4503 - accuracy: 0.8541 - val_loss: 0.2916 - val_accuracy: 0.8940
Epoch 7/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4339 - accuracy: 0.8574 - val_loss: 0.2848 - val_accura

In [107]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 54s 76ms/step - loss: 0.9052 - accuracy: 0.6821 - val_loss: 0.3903 - val_accuracy: 0.8586
Epoch 2/10
700/700 [==============================] - 53s 76ms/step - loss: 0.5677 - accuracy: 0.8098 - val_loss: 0.3159 - val_accuracy: 0.8801
Epoch 3/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4923 - accuracy: 0.8365 - val_loss: 0.2897 - val_accuracy: 0.8965
Epoch 4/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4550 - accuracy: 0.8499 - val_loss: 0.2727 - val_accuracy: 0.9035
Epoch 5/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4418 - accuracy: 0.8562 - val_loss: 0.3015 - val_accuracy: 0.8935
Epoch 6/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4328 - accuracy: 0.8610 - val_loss: 0.2696 - val_accuracy: 0.9050
Epoch 7/10
700/700 [==============================] - 54s 77ms/step - loss: 0.4296 - accuracy: 0.8624 - val_loss: 0.2830 - val_accura

In [108]:
time.sleep(400)

In [109]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 54s 71ms/step - loss: 0.8451 - accuracy: 0.7022 - val_loss: 0.3904 - val_accuracy: 0.8549
Epoch 2/10
744/744 [==============================] - 51s 69ms/step - loss: 0.5282 - accuracy: 0.8211 - val_loss: 0.3281 - val_accuracy: 0.8852
Epoch 3/10
744/744 [==============================] - 54s 72ms/step - loss: 0.4626 - accuracy: 0.8464 - val_loss: 0.2918 - val_accuracy: 0.8924
Epoch 4/10
744/744 [==============================] - 51s 68ms/step - loss: 0.4280 - accuracy: 0.8589 - val_loss: 0.2751 - val_accuracy: 0.9026
Epoch 5/10
744/744 [==============================] - 49s 66ms/step - loss: 0.4091 - accuracy: 0.8643 - val_loss: 0.3112 - val_accuracy: 0.8919
Epoch 6/10
744/744 [==============================] - 49s 66ms/step - loss: 0.4037 - accuracy: 0.8689 - val_loss: 0.2685 - val_accuracy: 0.9057
Epoch 7/10
744/744 [==============================] - 51s 68ms/step - loss: 0.4100 - accuracy: 0.8688 - val_loss: 0.2815 - val_accura

In [110]:
time.sleep(400)

In [111]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 55s 68ms/step - loss: 0.7860 - accuracy: 0.7265 - val_loss: 0.3748 - val_accuracy: 0.8636
Epoch 2/10
788/788 [==============================] - 54s 68ms/step - loss: 0.4938 - accuracy: 0.8347 - val_loss: 0.3058 - val_accuracy: 0.8885
Epoch 3/10
788/788 [==============================] - 54s 68ms/step - loss: 0.4361 - accuracy: 0.8557 - val_loss: 0.2927 - val_accuracy: 0.8934
Epoch 4/10
788/788 [==============================] - 54s 69ms/step - loss: 0.4112 - accuracy: 0.8659 - val_loss: 0.2845 - val_accuracy: 0.9001
Epoch 5/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3976 - accuracy: 0.8709 - val_loss: 0.3073 - val_accuracy: 0.9019
Epoch 6/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3937 - accuracy: 0.8740 - val_loss: 0.2904 - val_accuracy: 0.8971
Epoch 7/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3953 - accuracy: 0.8742 - val_loss: 0.3181 - val_accura

In [112]:
time.sleep(400)

In [113]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 58s 68ms/step - loss: 0.7716 - accuracy: 0.7266 - val_loss: 0.3857 - val_accuracy: 0.8629
Epoch 2/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4790 - accuracy: 0.8377 - val_loss: 0.3375 - val_accuracy: 0.8752
Epoch 3/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4230 - accuracy: 0.8603 - val_loss: 0.3213 - val_accuracy: 0.8886
Epoch 4/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4028 - accuracy: 0.8681 - val_loss: 0.2914 - val_accuracy: 0.8952
Epoch 5/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3876 - accuracy: 0.8732 - val_loss: 0.2845 - val_accuracy: 0.9006
Epoch 6/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3838 - accuracy: 0.8761 - val_loss: 0.2885 - val_accuracy: 0.8984
Epoch 7/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3839 - accuracy: 0.8752 - val_loss: 0.2804 - val_accura

In [114]:
time.sleep(400)

In [115]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 59s 66ms/step - loss: 0.7134 - accuracy: 0.7499 - val_loss: 0.3788 - val_accuracy: 0.8598
Epoch 2/10
875/875 [==============================] - 59s 67ms/step - loss: 0.4468 - accuracy: 0.8508 - val_loss: 0.3112 - val_accuracy: 0.8868
Epoch 3/10
875/875 [==============================] - 58s 66ms/step - loss: 0.3958 - accuracy: 0.8694 - val_loss: 0.2896 - val_accuracy: 0.8948
Epoch 4/10
875/875 [==============================] - 58s 67ms/step - loss: 0.3756 - accuracy: 0.8770 - val_loss: 0.2838 - val_accuracy: 0.8966
Epoch 5/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3648 - accuracy: 0.8814 - val_loss: 0.2880 - val_accuracy: 0.8944
Epoch 6/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3628 - accuracy: 0.8838 - val_loss: 0.3080 - val_accuracy: 0.8981
Epoch 7/10
875/875 [==============================] - 60s 69ms/step - loss: 0.3621 - accuracy: 0.8852 - val_loss: 0.2893 - val_accura

In [116]:
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_dg_e1"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_dg_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [117]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [118]:
import gc
gc.collect()

89881

### Softmax 

In [119]:
#softmax values
se_direction = "D:/guided-retraining/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [120]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [121]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [122]:
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_se.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [123]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 150ms/step - loss: 1.9257 - accuracy: 0.2686 - val_loss: 2.1967 - val_accuracy: 0.1976
Epoch 2/10
44/44 [==============================] - 7s 151ms/step - loss: 1.7486 - accuracy: 0.3064 - val_loss: 1.9976 - val_accuracy: 0.2734
Epoch 3/10
44/44 [==============================] - 6s 147ms/step - loss: 1.6588 - accuracy: 0.3229 - val_loss: 1.9463 - val_accuracy: 0.3512
Epoch 4/10
44/44 [==============================] - 6s 146ms/step - loss: 1.6029 - accuracy: 0.3518 - val_loss: 1.8095 - val_accuracy: 0.4104
Epoch 5/10
44/44 [==============================] - 7s 152ms/step - loss: 1.5278 - accuracy: 0.3689 - val_loss: 1.6251 - val_accuracy: 0.4654
Epoch 6/10
44/44 [==============================] - 7s 153ms/step - loss: 1.4790 - accuracy: 0.3907 - val_loss: 1.5533 - val_accuracy: 0.4620
Epoch 7/10
44/44 [==============================] - 7s 161ms/step - loss: 1.4571 - accuracy: 0.3989 - val_loss: 1.6140 - val_accuracy: 0.4807
Epoc

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 11s 111ms/step - loss: 1.8150 - accuracy: 0.2818 - val_loss: 1.9546 - val_accuracy: 0.3190
Epoch 2/10
88/88 [==============================] - 10s 111ms/step - loss: 1.5295 - accuracy: 0.3663 - val_loss: 1.8295 - val_accuracy: 0.3435
Epoch 3/10
88/88 [==============================] - 10s 111ms/step - loss: 1.4266 - accuracy: 0.3918 - val_loss: 1.6637 - val_accuracy: 0.4196
Epoch 4/10
88/88 [==============================] - 10s 114ms/step - loss: 1.3654 - accuracy: 0.4223 - val_loss: 1.5720 - val_accuracy: 0.4561
Epoch 5/10
88/88 [==============================] - 10s 110ms/step - loss: 1.3170 - accuracy: 0.4264 - val_loss: 1.4876 - val_accuracy: 0.5003
Epoch 6/10
88/88 [==============================] - 10s 112ms/step - loss: 1.2723 - accuracy: 0.4464 - val_loss: 1.4158 - val_accuracy: 0.5252
Epoch 7/10
88/88 [==============================] - 10s 114ms/step - loss: 1.2437 - accuracy: 0.4655 - val_loss: 1.3540 - val_accuracy: 0.48

In [125]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 14s 100ms/step - loss: 1.6619 - accuracy: 0.3410 - val_loss: 1.7984 - val_accuracy: 0.4689
Epoch 2/10
132/132 [==============================] - 13s 98ms/step - loss: 1.3640 - accuracy: 0.4317 - val_loss: 1.6418 - val_accuracy: 0.4531
Epoch 3/10
132/132 [==============================] - 14s 106ms/step - loss: 1.2693 - accuracy: 0.4586 - val_loss: 1.4674 - val_accuracy: 0.4836
Epoch 4/10
132/132 [==============================] - 14s 103ms/step - loss: 1.2141 - accuracy: 0.4721 - val_loss: 1.5016 - val_accuracy: 0.4526
Epoch 5/10
132/132 [==============================] - 14s 106ms/step - loss: 1.1632 - accuracy: 0.4951 - val_loss: 1.3837 - val_accuracy: 0.4914
Epoch 6/10
132/132 [==============================] - 15s 113ms/step - loss: 1.1308 - accuracy: 0.5050 - val_loss: 1.3371 - val_accuracy: 0.5303
Epoch 7/10
132/132 [==============================] - 14s 106ms/step - loss: 1.1042 - accuracy: 0.5192 - val_loss: 1.1825 - val_a

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 18s 97ms/step - loss: 1.5689 - accuracy: 0.3746 - val_loss: 1.6466 - val_accuracy: 0.4607
Epoch 2/10
175/175 [==============================] - 17s 96ms/step - loss: 1.2749 - accuracy: 0.4671 - val_loss: 1.6217 - val_accuracy: 0.5050
Epoch 3/10
175/175 [==============================] - 17s 97ms/step - loss: 1.1786 - accuracy: 0.4966 - val_loss: 1.4143 - val_accuracy: 0.5027
Epoch 4/10
175/175 [==============================] - 17s 95ms/step - loss: 1.1348 - accuracy: 0.5098 - val_loss: 1.3753 - val_accuracy: 0.4854
Epoch 5/10
175/175 [==============================] - 17s 96ms/step - loss: 1.0768 - accuracy: 0.5354 - val_loss: 1.2650 - val_accuracy: 0.5333
Epoch 6/10
175/175 [==============================] - 17s 98ms/step - loss: 1.0448 - accuracy: 0.5539 - val_loss: 1.2492 - val_accuracy: 0.5499
Epoch 7/10
175/175 [==============================] - 17s 97ms/step - loss: 1.0093 - accuracy: 0.5683 - val_loss: 1.2352 - val_accurac

In [127]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 21s 92ms/step - loss: 1.5121 - accuracy: 0.3961 - val_loss: 1.6422 - val_accuracy: 0.4790
Epoch 2/10
219/219 [==============================] - 20s 91ms/step - loss: 1.2065 - accuracy: 0.4867 - val_loss: 1.6129 - val_accuracy: 0.5238
Epoch 3/10
219/219 [==============================] - 20s 90ms/step - loss: 1.1128 - accuracy: 0.5264 - val_loss: 1.4593 - val_accuracy: 0.5197
Epoch 4/10
219/219 [==============================] - 20s 90ms/step - loss: 1.0493 - accuracy: 0.5521 - val_loss: 1.3516 - val_accuracy: 0.5561
Epoch 5/10
219/219 [==============================] - 20s 91ms/step - loss: 1.0022 - accuracy: 0.5809 - val_loss: 1.2291 - val_accuracy: 0.5726
Epoch 6/10
219/219 [==============================] - 20s 91ms/step - loss: 0.9511 - accuracy: 0.6029 - val_loss: 1.2346 - val_accuracy: 0.5825
Epoch 7/10
219/219 [==============================] - 20s 90ms/step - loss: 0.9292 - accuracy: 0.6164 - val_loss: 1.2011 - val_accurac

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 24s 89ms/step - loss: 1.3937 - accuracy: 0.4311 - val_loss: 1.5954 - val_accuracy: 0.5152
Epoch 2/10
263/263 [==============================] - 23s 87ms/step - loss: 1.1214 - accuracy: 0.5255 - val_loss: 1.3981 - val_accuracy: 0.5288
Epoch 3/10
263/263 [==============================] - 23s 89ms/step - loss: 1.0334 - accuracy: 0.5713 - val_loss: 1.2977 - val_accuracy: 0.5659
Epoch 4/10
263/263 [==============================] - 23s 87ms/step - loss: 0.9700 - accuracy: 0.6061 - val_loss: 1.1875 - val_accuracy: 0.5835
Epoch 5/10
263/263 [==============================] - 23s 86ms/step - loss: 0.9216 - accuracy: 0.6328 - val_loss: 1.2014 - val_accuracy: 0.5953
Epoch 6/10
263/263 [==============================] - 23s 87ms/step - loss: 0.8847 - accuracy: 0.6473 - val_loss: 1.1021 - val_accuracy: 0.5953
Epoch 7/10
263/263 [==============================] - 23s 86ms/step - loss: 0.8440 - accuracy: 0.6667 - val_loss: 1.0425 - val_accurac

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 27s 84ms/step - loss: 1.4029 - accuracy: 0.4405 - val_loss: 1.7059 - val_accuracy: 0.5264
Epoch 2/10
307/307 [==============================] - 26s 83ms/step - loss: 1.0842 - accuracy: 0.5616 - val_loss: 1.3774 - val_accuracy: 0.5454
Epoch 3/10
307/307 [==============================] - 26s 84ms/step - loss: 0.9828 - accuracy: 0.6101 - val_loss: 1.2305 - val_accuracy: 0.5914
Epoch 4/10
307/307 [==============================] - 26s 84ms/step - loss: 0.9231 - accuracy: 0.6377 - val_loss: 1.2269 - val_accuracy: 0.5656
Epoch 5/10
307/307 [==============================] - 26s 83ms/step - loss: 0.8571 - accuracy: 0.6702 - val_loss: 1.0766 - val_accuracy: 0.5959
Epoch 6/10
307/307 [==============================] - 26s 84ms/step - loss: 0.8222 - accuracy: 0.6885 - val_loss: 1.0230 - val_accuracy: 0.6091
Epoch 7/10
307/307 [==============================] - 26s 84ms/step - loss: 0.7835 - accuracy: 0.7087 - val_loss: 0.9987 - val_accurac

In [130]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 29s 81ms/step - loss: 1.2957 - accuracy: 0.4851 - val_loss: 1.3148 - val_accuracy: 0.5428
Epoch 2/10
350/350 [==============================] - 28s 81ms/step - loss: 0.9772 - accuracy: 0.6224 - val_loss: 1.2582 - val_accuracy: 0.5758
Epoch 3/10
350/350 [==============================] - 28s 81ms/step - loss: 0.8768 - accuracy: 0.6746 - val_loss: 0.9971 - val_accuracy: 0.6032
Epoch 4/10
350/350 [==============================] - 28s 81ms/step - loss: 0.7980 - accuracy: 0.7098 - val_loss: 1.0057 - val_accuracy: 0.6146
Epoch 5/10
350/350 [==============================] - 28s 80ms/step - loss: 0.7490 - accuracy: 0.7275 - val_loss: 0.9857 - val_accuracy: 0.6419
Epoch 6/10
350/350 [==============================] - 28s 80ms/step - loss: 0.7119 - accuracy: 0.7438 - val_loss: 0.8000 - val_accuracy: 0.7358
Epoch 7/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6815 - accuracy: 0.7540 - val_loss: 0.8144 - val_accurac

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 33s 81ms/step - loss: 1.2615 - accuracy: 0.5199 - val_loss: 1.3805 - val_accuracy: 0.5671
Epoch 2/10
394/394 [==============================] - 34s 85ms/step - loss: 0.9131 - accuracy: 0.6682 - val_loss: 1.1548 - val_accuracy: 0.5837
Epoch 3/10
394/394 [==============================] - 32s 82ms/step - loss: 0.8018 - accuracy: 0.7116 - val_loss: 1.0594 - val_accuracy: 0.6182
Epoch 4/10
394/394 [==============================] - 32s 81ms/step - loss: 0.7387 - accuracy: 0.7383 - val_loss: 0.9732 - val_accuracy: 0.6354
Epoch 5/10
394/394 [==============================] - 32s 81ms/step - loss: 0.7013 - accuracy: 0.7546 - val_loss: 0.9375 - val_accuracy: 0.6441
Epoch 6/10
394/394 [==============================] - 32s 81ms/step - loss: 0.6694 - accuracy: 0.7634 - val_loss: 1.0235 - val_accuracy: 0.6331
Epoch 7/10
394/394 [==============================] - 32s 81ms/step - loss: 0.6414 - accuracy: 0.7802 - val_loss: 0.7444 - val_accurac

In [132]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 37s 81ms/step - loss: 1.1491 - accuracy: 0.5663 - val_loss: 1.1931 - val_accuracy: 0.5666
Epoch 2/10
438/438 [==============================] - 35s 81ms/step - loss: 0.8229 - accuracy: 0.7054 - val_loss: 0.9253 - val_accuracy: 0.6203
Epoch 3/10
438/438 [==============================] - 35s 81ms/step - loss: 0.7250 - accuracy: 0.7470 - val_loss: 0.8509 - val_accuracy: 0.6477
Epoch 4/10
438/438 [==============================] - 35s 80ms/step - loss: 0.6781 - accuracy: 0.7642 - val_loss: 0.8220 - val_accuracy: 0.6611
Epoch 5/10
438/438 [==============================] - 35s 80ms/step - loss: 0.6428 - accuracy: 0.7787 - val_loss: 0.8521 - val_accuracy: 0.7082
Epoch 6/10
438/438 [==============================] - 35s 80ms/step - loss: 0.6186 - accuracy: 0.7909 - val_loss: 0.7192 - val_accuracy: 0.7495
Epoch 7/10
438/438 [==============================] - 34s 79ms/step - loss: 0.5977 - accuracy: 0.7946 - val_loss: 0.6789 - val_accurac

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 39s 79ms/step - loss: 1.0876 - accuracy: 0.6045 - val_loss: 1.1207 - val_accuracy: 0.5791
Epoch 2/10
482/482 [==============================] - 39s 80ms/step - loss: 0.7518 - accuracy: 0.7370 - val_loss: 0.8300 - val_accuracy: 0.6800
Epoch 3/10
482/482 [==============================] - 39s 81ms/step - loss: 0.6707 - accuracy: 0.7703 - val_loss: 0.7441 - val_accuracy: 0.7344
Epoch 4/10
482/482 [==============================] - 38s 79ms/step - loss: 0.6167 - accuracy: 0.7906 - val_loss: 0.7188 - val_accuracy: 0.7669
Epoch 5/10
482/482 [==============================] - 38s 79ms/step - loss: 0.5886 - accuracy: 0.7981 - val_loss: 0.6263 - val_accuracy: 0.7651
Epoch 6/10
482/482 [==============================] - 38s 79ms/step - loss: 0.5716 - accuracy: 0.8061 - val_loss: 0.5822 - val_accuracy: 0.7834
Epoch 7/10
482/482 [==============================] - 38s 79ms/step - loss: 0.5538 - accuracy: 0.8152 - val_loss: 0.5647 - val_accura

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 43s 80ms/step - loss: 1.0759 - accuracy: 0.6095 - val_loss: 0.9534 - val_accuracy: 0.6153
Epoch 2/10
525/525 [==============================] - 42s 79ms/step - loss: 0.7367 - accuracy: 0.7460 - val_loss: 0.7843 - val_accuracy: 0.6488
Epoch 3/10
525/525 [==============================] - 41s 77ms/step - loss: 0.6490 - accuracy: 0.7786 - val_loss: 0.6398 - val_accuracy: 0.7615
Epoch 4/10
525/525 [==============================] - 41s 79ms/step - loss: 0.5975 - accuracy: 0.7966 - val_loss: 0.5650 - val_accuracy: 0.7922
Epoch 5/10
525/525 [==============================] - 42s 80ms/step - loss: 0.5724 - accuracy: 0.8056 - val_loss: 0.5334 - val_accuracy: 0.8064
Epoch 6/10
525/525 [==============================] - 41s 78ms/step - loss: 0.5451 - accuracy: 0.8185 - val_loss: 0.5450 - val_accuracy: 0.8180
Epoch 7/10
525/525 [==============================] - 41s 78ms/step - loss: 0.5325 - accuracy: 0.8212 - val_loss: 0.4745 - val_accura

In [135]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 45s 78ms/step - loss: 1.0113 - accuracy: 0.6434 - val_loss: 0.7878 - val_accuracy: 0.6399
Epoch 2/10
569/569 [==============================] - 44s 77ms/step - loss: 0.6960 - accuracy: 0.7609 - val_loss: 0.5588 - val_accuracy: 0.7849
Epoch 3/10
569/569 [==============================] - 44s 78ms/step - loss: 0.6156 - accuracy: 0.7926 - val_loss: 0.4718 - val_accuracy: 0.8312
Epoch 4/10
569/569 [==============================] - 44s 77ms/step - loss: 0.5711 - accuracy: 0.8065 - val_loss: 0.4670 - val_accuracy: 0.8418
Epoch 5/10
569/569 [==============================] - 44s 77ms/step - loss: 0.5399 - accuracy: 0.8202 - val_loss: 0.3493 - val_accuracy: 0.8855
Epoch 6/10
569/569 [==============================] - 44s 77ms/step - loss: 0.5216 - accuracy: 0.8265 - val_loss: 0.3749 - val_accuracy: 0.8720
Epoch 7/10
569/569 [==============================] - 44s 77ms/step - loss: 0.5084 - accuracy: 0.8309 - val_loss: 0.3280 - val_accura

In [136]:
n=13
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 50s 79ms/step - loss: 0.9763 - accuracy: 0.6557 - val_loss: 0.6411 - val_accuracy: 0.7625
Epoch 2/10
613/613 [==============================] - 50s 82ms/step - loss: 0.6579 - accuracy: 0.7715 - val_loss: 0.4435 - val_accuracy: 0.8425
Epoch 3/10
613/613 [==============================] - 49s 79ms/step - loss: 0.5714 - accuracy: 0.8071 - val_loss: 0.3580 - val_accuracy: 0.8838
Epoch 4/10
613/613 [==============================] - 49s 80ms/step - loss: 0.5313 - accuracy: 0.8245 - val_loss: 0.3070 - val_accuracy: 0.8903
Epoch 5/10
613/613 [==============================] - 49s 80ms/step - loss: 0.5017 - accuracy: 0.8326 - val_loss: 0.2883 - val_accuracy: 0.8981
Epoch 6/10
613/613 [==============================] - 48s 79ms/step - loss: 0.4787 - accuracy: 0.8410 - val_loss: 0.2806 - val_accuracy: 0.8997
Epoch 7/10
613/613 [==============================] - 47s 76ms/step - loss: 0.4760 - accuracy: 0.8432 - val_loss: 0.2839 - val_accura

In [137]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 53s 80ms/step - loss: 0.9648 - accuracy: 0.6623 - val_loss: 0.5096 - val_accuracy: 0.8176
Epoch 2/10
657/657 [==============================] - 52s 78ms/step - loss: 0.6315 - accuracy: 0.7872 - val_loss: 0.3413 - val_accuracy: 0.8740
Epoch 3/10
657/657 [==============================] - 52s 79ms/step - loss: 0.5385 - accuracy: 0.8201 - val_loss: 0.3048 - val_accuracy: 0.8864
Epoch 4/10
657/657 [==============================] - 52s 78ms/step - loss: 0.4945 - accuracy: 0.8336 - val_loss: 0.2857 - val_accuracy: 0.8953
Epoch 5/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4592 - accuracy: 0.8470 - val_loss: 0.2759 - val_accuracy: 0.9009
Epoch 6/10
657/657 [==============================] - 51s 78ms/step - loss: 0.4524 - accuracy: 0.8519 - val_loss: 0.2731 - val_accuracy: 0.9042
Epoch 7/10
657/657 [==============================] - 51s 78ms/step - loss: 0.4399 - accuracy: 0.8576 - val_loss: 0.3488 - val_accura

In [138]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 55s 78ms/step - loss: 0.8786 - accuracy: 0.6904 - val_loss: 0.3956 - val_accuracy: 0.8570
Epoch 2/10
700/700 [==============================] - 53s 76ms/step - loss: 0.5634 - accuracy: 0.8125 - val_loss: 0.3215 - val_accuracy: 0.8810
Epoch 3/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4839 - accuracy: 0.8404 - val_loss: 0.2785 - val_accuracy: 0.9000
Epoch 4/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4459 - accuracy: 0.8510 - val_loss: 0.2706 - val_accuracy: 0.9010
Epoch 5/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4336 - accuracy: 0.8592 - val_loss: 0.2792 - val_accuracy: 0.8991
Epoch 6/10
700/700 [==============================] - 53s 75ms/step - loss: 0.4170 - accuracy: 0.8606 - val_loss: 0.2655 - val_accuracy: 0.9049
Epoch 7/10
700/700 [==============================] - 53s 75ms/step - loss: 0.4140 - accuracy: 0.8663 - val_loss: 0.2840 - val_accura

In [139]:
time.sleep(400)

In [140]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 54s 71ms/step - loss: 0.8545 - accuracy: 0.6976 - val_loss: 0.3769 - val_accuracy: 0.8643
Epoch 2/10
744/744 [==============================] - 53s 72ms/step - loss: 0.5348 - accuracy: 0.8228 - val_loss: 0.3122 - val_accuracy: 0.8859
Epoch 3/10
744/744 [==============================] - 52s 70ms/step - loss: 0.4676 - accuracy: 0.8450 - val_loss: 0.2916 - val_accuracy: 0.8984
Epoch 4/10
744/744 [==============================] - 52s 69ms/step - loss: 0.4309 - accuracy: 0.8589 - val_loss: 0.2838 - val_accuracy: 0.8999
Epoch 5/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4201 - accuracy: 0.8629 - val_loss: 0.2805 - val_accuracy: 0.9005
Epoch 6/10
744/744 [==============================] - 51s 68ms/step - loss: 0.4124 - accuracy: 0.8679 - val_loss: 0.2706 - val_accuracy: 0.9016
Epoch 7/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4136 - accuracy: 0.8689 - val_loss: 0.2761 - val_accura

In [141]:
time.sleep(400)

In [142]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 51s 63ms/step - loss: 0.8371 - accuracy: 0.7009 - val_loss: 0.4094 - val_accuracy: 0.8502
Epoch 2/10
788/788 [==============================] - 50s 64ms/step - loss: 0.5088 - accuracy: 0.8278 - val_loss: 0.3531 - val_accuracy: 0.8748
Epoch 3/10
788/788 [==============================] - 50s 64ms/step - loss: 0.4487 - accuracy: 0.8500 - val_loss: 0.3005 - val_accuracy: 0.8934
Epoch 4/10
788/788 [==============================] - 48s 61ms/step - loss: 0.4164 - accuracy: 0.8623 - val_loss: 0.3074 - val_accuracy: 0.8904
Epoch 5/10
788/788 [==============================] - 48s 61ms/step - loss: 0.4075 - accuracy: 0.8680 - val_loss: 0.3219 - val_accuracy: 0.8959
Epoch 6/10
788/788 [==============================] - 47s 60ms/step - loss: 0.4001 - accuracy: 0.8722 - val_loss: 0.2944 - val_accuracy: 0.9009
Epoch 7/10
788/788 [==============================] - 47s 60ms/step - loss: 0.3976 - accuracy: 0.8731 - val_loss: 0.3221 - val_accura

In [143]:
time.sleep(400)

In [144]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 52s 62ms/step - loss: 0.7674 - accuracy: 0.7311 - val_loss: 0.4047 - val_accuracy: 0.8520
Epoch 2/10
832/832 [==============================] - 52s 63ms/step - loss: 0.4789 - accuracy: 0.8403 - val_loss: 0.3205 - val_accuracy: 0.8864
Epoch 3/10
832/832 [==============================] - 51s 61ms/step - loss: 0.4132 - accuracy: 0.8624 - val_loss: 0.2944 - val_accuracy: 0.8964
Epoch 4/10
832/832 [==============================] - 51s 62ms/step - loss: 0.3934 - accuracy: 0.8732 - val_loss: 0.3239 - val_accuracy: 0.8970
Epoch 5/10
832/832 [==============================] - 49s 59ms/step - loss: 0.3821 - accuracy: 0.8758 - val_loss: 0.3242 - val_accuracy: 0.8996
Epoch 6/10
832/832 [==============================] - 49s 59ms/step - loss: 0.3798 - accuracy: 0.8787 - val_loss: 0.2915 - val_accuracy: 0.8994
Epoch 7/10
832/832 [==============================] - 50s 60ms/step - loss: 0.3767 - accuracy: 0.8803 - val_loss: 0.3316 - val_accura

In [145]:
time.sleep(400)

In [146]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 53s 60ms/step - loss: 0.7241 - accuracy: 0.7428 - val_loss: 0.3854 - val_accuracy: 0.8638
Epoch 2/10
875/875 [==============================] - 52s 60ms/step - loss: 0.4487 - accuracy: 0.8489 - val_loss: 0.3119 - val_accuracy: 0.8833
Epoch 3/10
875/875 [==============================] - 52s 59ms/step - loss: 0.3977 - accuracy: 0.8697 - val_loss: 0.2828 - val_accuracy: 0.8971
Epoch 4/10
875/875 [==============================] - 53s 60ms/step - loss: 0.3722 - accuracy: 0.8792 - val_loss: 0.2918 - val_accuracy: 0.8984
Epoch 5/10
875/875 [==============================] - 58s 67ms/step - loss: 0.3594 - accuracy: 0.8858 - val_loss: 0.2945 - val_accuracy: 0.8968
Epoch 6/10
875/875 [==============================] - 61s 69ms/step - loss: 0.3589 - accuracy: 0.8850 - val_loss: 0.2972 - val_accuracy: 0.8956
Epoch 7/10
875/875 [==============================] - 61s 70ms/step - loss: 0.3637 - accuracy: 0.8872 - val_loss: 0.2915 - val_accura

In [147]:
len(models_se)

20

In [148]:
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [149]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [150]:
gc.collect()

78274

## Training guided by Random values

In [18]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

[43703, 34094, 54014, 19868, 43544, 18605, 53966, 20381, 5989, 10792]
55998


In [19]:
save_dir = "D:/guided-retraining/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [20]:
len(random_indexes)

55998

In [21]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [22]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [23]:
len(image_sets_random)

20

In [66]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_random.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [40]:
#models_random[0].evaluate(x_test_and_adversary,y_test_and_adversary)

In [41]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 6s 125ms/step - loss: 1.7979 - accuracy: 0.3364 - val_loss: 0.9375 - val_accuracy: 0.7007
Epoch 2/10
44/44 [==============================] - 5s 120ms/step - loss: 1.1753 - accuracy: 0.5946 - val_loss: 0.7113 - val_accuracy: 0.7587
Epoch 3/10
44/44 [==============================] - 5s 120ms/step - loss: 0.9541 - accuracy: 0.6636 - val_loss: 0.6012 - val_accuracy: 0.7811
Epoch 4/10
44/44 [==============================] - 5s 125ms/step - loss: 0.8366 - accuracy: 0.7100 - val_loss: 0.5722 - val_accuracy: 0.7862
Epoch 5/10
44/44 [==============================] - 6s 129ms/step - loss: 0.7682 - accuracy: 0.7343 - val_loss: 0.5538 - val_accuracy: 0.7926
Epoch 6/10
44/44 [==============================] - 5s 126ms/step - loss: 0.6788 - accuracy: 0.7561 - val_loss: 0.5234 - val_accuracy: 0.8057
Epoch 7/10
44/44 [==============================] - 6s 129ms/step - loss: 0.6433 - accuracy: 0.7625 - val_loss: 0.5200 - val_accuracy: 0.8114
Epoc

In [42]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 9s 94ms/step - loss: 1.4871 - accuracy: 0.4582 - val_loss: 0.8110 - val_accuracy: 0.7110
Epoch 2/10
88/88 [==============================] - 8s 94ms/step - loss: 0.9249 - accuracy: 0.6777 - val_loss: 0.6209 - val_accuracy: 0.7743
Epoch 3/10
88/88 [==============================] - 8s 91ms/step - loss: 0.7587 - accuracy: 0.7343 - val_loss: 0.5208 - val_accuracy: 0.8035
Epoch 4/10
88/88 [==============================] - 8s 87ms/step - loss: 0.6641 - accuracy: 0.7573 - val_loss: 0.4774 - val_accuracy: 0.8100
Epoch 5/10
88/88 [==============================] - 8s 89ms/step - loss: 0.6018 - accuracy: 0.7839 - val_loss: 0.4828 - val_accuracy: 0.8238
Epoch 6/10
88/88 [==============================] - 7s 84ms/step - loss: 0.5540 - accuracy: 0.7925 - val_loss: 0.4383 - val_accuracy: 0.8349
Epoch 7/10
88/88 [==============================] - 8s 90ms/step - loss: 0.5283 - accuracy: 0.8121 - val_loss: 0.4501 - val_accuracy: 0.8225
Epoch 8/10


In [43]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 11s 78ms/step - loss: 1.3501 - accuracy: 0.5213 - val_loss: 0.7359 - val_accuracy: 0.7259
Epoch 2/10
132/132 [==============================] - 10s 75ms/step - loss: 0.8201 - accuracy: 0.7132 - val_loss: 0.5366 - val_accuracy: 0.8031
Epoch 3/10
132/132 [==============================] - 10s 76ms/step - loss: 0.6739 - accuracy: 0.7658 - val_loss: 0.4591 - val_accuracy: 0.8256
Epoch 4/10
132/132 [==============================] - 10s 77ms/step - loss: 0.5953 - accuracy: 0.7886 - val_loss: 0.4349 - val_accuracy: 0.8439
Epoch 5/10
132/132 [==============================] - 10s 78ms/step - loss: 0.5538 - accuracy: 0.8073 - val_loss: 0.4123 - val_accuracy: 0.8462
Epoch 6/10
132/132 [==============================] - 10s 77ms/step - loss: 0.5123 - accuracy: 0.8267 - val_loss: 0.3956 - val_accuracy: 0.8546
Epoch 7/10
132/132 [==============================] - 10s 77ms/step - loss: 0.4703 - accuracy: 0.8332 - val_loss: 0.3747 - val_accurac

In [44]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 14s 73ms/step - loss: 1.1924 - accuracy: 0.5748 - val_loss: 0.5803 - val_accuracy: 0.7807
Epoch 2/10
175/175 [==============================] - 12s 72ms/step - loss: 0.7349 - accuracy: 0.7408 - val_loss: 0.4816 - val_accuracy: 0.8131
Epoch 3/10
175/175 [==============================] - 13s 72ms/step - loss: 0.6228 - accuracy: 0.7852 - val_loss: 0.4466 - val_accuracy: 0.8328
Epoch 4/10
175/175 [==============================] - 13s 75ms/step - loss: 0.5555 - accuracy: 0.8079 - val_loss: 0.4106 - val_accuracy: 0.8504
Epoch 5/10
175/175 [==============================] - 13s 75ms/step - loss: 0.5114 - accuracy: 0.8266 - val_loss: 0.3926 - val_accuracy: 0.8546
Epoch 6/10
175/175 [==============================] - 13s 73ms/step - loss: 0.4654 - accuracy: 0.8375 - val_loss: 0.3583 - val_accuracy: 0.8715
Epoch 7/10
175/175 [==============================] - 12s 71ms/step - loss: 0.4470 - accuracy: 0.8501 - val_loss: 0.3631 - val_accurac

In [25]:
n=4
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 25s 103ms/step - loss: 1.1369 - accuracy: 0.6015 - val_loss: 0.5630 - val_accuracy: 0.7840
Epoch 2/10
219/219 [==============================] - 20s 93ms/step - loss: 0.6775 - accuracy: 0.7611 - val_loss: 0.4664 - val_accuracy: 0.8163
Epoch 3/10
219/219 [==============================] - 22s 99ms/step - loss: 0.5694 - accuracy: 0.8037 - val_loss: 0.4465 - val_accuracy: 0.8321
Epoch 4/10
219/219 [==============================] - 22s 100ms/step - loss: 0.5042 - accuracy: 0.8280 - val_loss: 0.3877 - val_accuracy: 0.8566
Epoch 5/10
219/219 [==============================] - 22s 100ms/step - loss: 0.4577 - accuracy: 0.8444 - val_loss: 0.3607 - val_accuracy: 0.8714
Epoch 6/10
219/219 [==============================] - 22s 100ms/step - loss: 0.4260 - accuracy: 0.8568 - val_loss: 0.3670 - val_accuracy: 0.8683
Epoch 7/10
219/219 [==============================] - 21s 98ms/step - loss: 0.4013 - accuracy: 0.8631 - val_loss: 0.3363 - val_acc

In [26]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 27s 97ms/step - loss: 1.0252 - accuracy: 0.6324 - val_loss: 0.5235 - val_accuracy: 0.8029
Epoch 2/10
263/263 [==============================] - 26s 98ms/step - loss: 0.6185 - accuracy: 0.7814 - val_loss: 0.4446 - val_accuracy: 0.8403
Epoch 3/10
263/263 [==============================] - 26s 98ms/step - loss: 0.5189 - accuracy: 0.8231 - val_loss: 0.4071 - val_accuracy: 0.8472
Epoch 4/10
263/263 [==============================] - 25s 96ms/step - loss: 0.4713 - accuracy: 0.8416 - val_loss: 0.3746 - val_accuracy: 0.8596
Epoch 5/10
263/263 [==============================] - 25s 93ms/step - loss: 0.4381 - accuracy: 0.8538 - val_loss: 0.3509 - val_accuracy: 0.8769
Epoch 6/10
263/263 [==============================] - 25s 96ms/step - loss: 0.4143 - accuracy: 0.8601 - val_loss: 0.3353 - val_accuracy: 0.8811
Epoch 7/10
263/263 [==============================] - 25s 96ms/step - loss: 0.3874 - accuracy: 0.8702 - val_loss: 0.3467 - val_accurac

In [33]:
n=6
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 20s 64ms/step - loss: 0.3407 - accuracy: 0.8884 - val_loss: 0.3046 - val_accuracy: 0.8924
Epoch 2/10
307/307 [==============================] - 20s 64ms/step - loss: 0.3308 - accuracy: 0.8909 - val_loss: 0.3061 - val_accuracy: 0.8954
Epoch 3/10
307/307 [==============================] - 21s 67ms/step - loss: 0.3303 - accuracy: 0.8898 - val_loss: 0.2987 - val_accuracy: 0.9014
Epoch 4/10
307/307 [==============================] - 21s 68ms/step - loss: 0.3352 - accuracy: 0.8919 - val_loss: 0.3232 - val_accuracy: 0.8918
Epoch 5/10
307/307 [==============================] - 21s 69ms/step - loss: 0.3299 - accuracy: 0.8914 - val_loss: 0.3045 - val_accuracy: 0.8979
Epoch 6/10
307/307 [==============================] - 20s 66ms/step - loss: 0.3301 - accuracy: 0.8931 - val_loss: 0.3172 - val_accuracy: 0.8901
Epoch 7/10
307/307 [==============================] - 21s 67ms/step - loss: 0.3185 - accuracy: 0.8956 - val_loss: 0.3689 - val_accurac

In [34]:
n=7
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 24s 68ms/step - loss: 0.3308 - accuracy: 0.8907 - val_loss: 0.2863 - val_accuracy: 0.9005
Epoch 2/10
350/350 [==============================] - 22s 64ms/step - loss: 0.3233 - accuracy: 0.8952 - val_loss: 0.2821 - val_accuracy: 0.9021
Epoch 3/10
350/350 [==============================] - 22s 64ms/step - loss: 0.3242 - accuracy: 0.8914 - val_loss: 0.3260 - val_accuracy: 0.8980
Epoch 4/10
350/350 [==============================] - 23s 65ms/step - loss: 0.3206 - accuracy: 0.8925 - val_loss: 0.3165 - val_accuracy: 0.9011
Epoch 5/10
350/350 [==============================] - 24s 68ms/step - loss: 0.3160 - accuracy: 0.8986 - val_loss: 0.2923 - val_accuracy: 0.8998
Epoch 6/10
350/350 [==============================] - 23s 67ms/step - loss: 0.3226 - accuracy: 0.8976 - val_loss: 0.3257 - val_accuracy: 0.8921
Epoch 7/10
350/350 [==============================] - 23s 66ms/step - loss: 0.3208 - accuracy: 0.8963 - val_loss: 0.3093 - val_accurac

In [29]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 38s 92ms/step - loss: 0.9475 - accuracy: 0.6648 - val_loss: 0.5252 - val_accuracy: 0.7994
Epoch 2/10
394/394 [==============================] - 37s 93ms/step - loss: 0.5669 - accuracy: 0.8038 - val_loss: 0.3926 - val_accuracy: 0.8548
Epoch 3/10
394/394 [==============================] - 36s 91ms/step - loss: 0.4779 - accuracy: 0.8409 - val_loss: 0.3416 - val_accuracy: 0.8721
Epoch 4/10
394/394 [==============================] - 36s 91ms/step - loss: 0.4269 - accuracy: 0.8555 - val_loss: 0.3342 - val_accuracy: 0.8788
Epoch 5/10
394/394 [==============================] - 36s 91ms/step - loss: 0.4002 - accuracy: 0.8649 - val_loss: 0.3250 - val_accuracy: 0.8836
Epoch 6/10
394/394 [==============================] - 36s 91ms/step - loss: 0.3825 - accuracy: 0.8720 - val_loss: 0.3011 - val_accuracy: 0.8933
Epoch 7/10
394/394 [==============================] - 36s 91ms/step - loss: 0.3648 - accuracy: 0.8776 - val_loss: 0.2980 - val_accurac

In [30]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 41s 89ms/step - loss: 0.8829 - accuracy: 0.6855 - val_loss: 0.4561 - val_accuracy: 0.8366
Epoch 2/10
438/438 [==============================] - 37s 84ms/step - loss: 0.5431 - accuracy: 0.8118 - val_loss: 0.3783 - val_accuracy: 0.8571
Epoch 3/10
438/438 [==============================] - 29s 67ms/step - loss: 0.4610 - accuracy: 0.8444 - val_loss: 0.3357 - val_accuracy: 0.8753
Epoch 4/10
438/438 [==============================] - 28s 63ms/step - loss: 0.4184 - accuracy: 0.8602 - val_loss: 0.3431 - val_accuracy: 0.8774
Epoch 5/10
438/438 [==============================] - 28s 63ms/step - loss: 0.3955 - accuracy: 0.8702 - val_loss: 0.3312 - val_accuracy: 0.8834
Epoch 6/10
438/438 [==============================] - 28s 64ms/step - loss: 0.3726 - accuracy: 0.8778 - val_loss: 0.3224 - val_accuracy: 0.8829
Epoch 7/10
438/438 [==============================] - 29s 66ms/step - loss: 0.3652 - accuracy: 0.8774 - val_loss: 0.3268 - val_accurac

In [31]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 31s 63ms/step - loss: 0.8609 - accuracy: 0.6935 - val_loss: 0.4874 - val_accuracy: 0.8172
Epoch 2/10
482/482 [==============================] - 29s 61ms/step - loss: 0.5161 - accuracy: 0.8235 - val_loss: 0.4061 - val_accuracy: 0.8485
Epoch 3/10
482/482 [==============================] - 30s 61ms/step - loss: 0.4428 - accuracy: 0.8526 - val_loss: 0.3362 - val_accuracy: 0.8761
Epoch 4/10
482/482 [==============================] - 30s 63ms/step - loss: 0.4003 - accuracy: 0.8668 - val_loss: 0.3279 - val_accuracy: 0.8851
Epoch 5/10
482/482 [==============================] - 31s 64ms/step - loss: 0.3792 - accuracy: 0.8737 - val_loss: 0.3283 - val_accuracy: 0.8841
Epoch 6/10
482/482 [==============================] - 31s 65ms/step - loss: 0.3695 - accuracy: 0.8787 - val_loss: 0.3066 - val_accuracy: 0.8909
Epoch 7/10
482/482 [==============================] - 31s 64ms/step - loss: 0.3564 - accuracy: 0.8824 - val_loss: 0.3012 - val_accura

In [32]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 35s 65ms/step - loss: 0.8291 - accuracy: 0.7013 - val_loss: 0.4307 - val_accuracy: 0.8471
Epoch 2/10
525/525 [==============================] - 32s 61ms/step - loss: 0.5022 - accuracy: 0.8299 - val_loss: 0.3691 - val_accuracy: 0.8691
Epoch 3/10
525/525 [==============================] - 32s 60ms/step - loss: 0.4330 - accuracy: 0.8558 - val_loss: 0.3373 - val_accuracy: 0.8771
Epoch 4/10
525/525 [==============================] - 32s 61ms/step - loss: 0.3925 - accuracy: 0.8706 - val_loss: 0.2959 - val_accuracy: 0.8901
Epoch 5/10
525/525 [==============================] - 34s 65ms/step - loss: 0.3749 - accuracy: 0.8768 - val_loss: 0.2888 - val_accuracy: 0.8990
Epoch 6/10
525/525 [==============================] - 33s 63ms/step - loss: 0.3595 - accuracy: 0.8807 - val_loss: 0.3037 - val_accuracy: 0.8997
Epoch 7/10
525/525 [==============================] - 35s 67ms/step - loss: 0.3522 - accuracy: 0.8838 - val_loss: 0.2997 - val_accura

In [67]:
n=12
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 36s 63ms/step - loss: 0.8285 - accuracy: 0.7039 - val_loss: 0.4253 - val_accuracy: 0.8382
Epoch 2/10
569/569 [==============================] - 36s 63ms/step - loss: 0.5042 - accuracy: 0.8277 - val_loss: 0.3822 - val_accuracy: 0.8658
Epoch 3/10
569/569 [==============================] - 35s 61ms/step - loss: 0.4307 - accuracy: 0.8562 - val_loss: 0.3161 - val_accuracy: 0.8854
Epoch 4/10
569/569 [==============================] - 35s 62ms/step - loss: 0.3994 - accuracy: 0.8675 - val_loss: 0.3219 - val_accuracy: 0.8786
Epoch 5/10
569/569 [==============================] - 35s 61ms/step - loss: 0.3773 - accuracy: 0.8757 - val_loss: 0.3012 - val_accuracy: 0.8957
Epoch 6/10
569/569 [==============================] - 35s 61ms/step - loss: 0.3728 - accuracy: 0.8784 - val_loss: 0.2915 - val_accuracy: 0.8963
Epoch 7/10
569/569 [==============================] - 35s 61ms/step - loss: 0.3597 - accuracy: 0.8833 - val_loss: 0.2880 - val_accura

In [68]:
#print(models_random[12].model.layers[-1].weights[0][0])
#print(models_random[14].model.layers[-1].weights[0][0])
#print(models_random[16].model.layers[-1].weights[0][0])

In [69]:
n=13
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 37s 59ms/step - loss: 0.7981 - accuracy: 0.7181 - val_loss: 0.4059 - val_accuracy: 0.8486
Epoch 2/10
613/613 [==============================] - 36s 59ms/step - loss: 0.4883 - accuracy: 0.8370 - val_loss: 0.3369 - val_accuracy: 0.8779
Epoch 3/10
613/613 [==============================] - 36s 59ms/step - loss: 0.4163 - accuracy: 0.8633 - val_loss: 0.3097 - val_accuracy: 0.8894
Epoch 4/10
613/613 [==============================] - 38s 61ms/step - loss: 0.3830 - accuracy: 0.8722 - val_loss: 0.2965 - val_accuracy: 0.8929
Epoch 5/10
613/613 [==============================] - 38s 62ms/step - loss: 0.3624 - accuracy: 0.8803 - val_loss: 0.3965 - val_accuracy: 0.8891
Epoch 6/10
613/613 [==============================] - 36s 59ms/step - loss: 0.3522 - accuracy: 0.8852 - val_loss: 0.2879 - val_accuracy: 0.8983
Epoch 7/10
613/613 [==============================] - 41s 68ms/step - loss: 0.3555 - accuracy: 0.8857 - val_loss: 0.2969 - val_accura

In [70]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 44s 66ms/step - loss: 0.7816 - accuracy: 0.7330 - val_loss: 0.4135 - val_accuracy: 0.8508
Epoch 2/10
657/657 [==============================] - 44s 66ms/step - loss: 0.4667 - accuracy: 0.8425 - val_loss: 0.4008 - val_accuracy: 0.8550
Epoch 3/10
657/657 [==============================] - 42s 64ms/step - loss: 0.4118 - accuracy: 0.8642 - val_loss: 0.3174 - val_accuracy: 0.8831
Epoch 4/10
657/657 [==============================] - 46s 70ms/step - loss: 0.3820 - accuracy: 0.8745 - val_loss: 0.3333 - val_accuracy: 0.8864
Epoch 5/10
657/657 [==============================] - 45s 69ms/step - loss: 0.3594 - accuracy: 0.8813 - val_loss: 0.3134 - val_accuracy: 0.8882
Epoch 6/10
657/657 [==============================] - 53s 81ms/step - loss: 0.3608 - accuracy: 0.8834 - val_loss: 0.3145 - val_accuracy: 0.8956
Epoch 7/10
657/657 [==============================] - 45s 69ms/step - loss: 0.3542 - accuracy: 0.8862 - val_loss: 0.3413 - val_accura

In [71]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 45s 63ms/step - loss: 0.7469 - accuracy: 0.7340 - val_loss: 0.3981 - val_accuracy: 0.8526
Epoch 2/10
700/700 [==============================] - 44s 62ms/step - loss: 0.4598 - accuracy: 0.8460 - val_loss: 0.3347 - val_accuracy: 0.8781
Epoch 3/10
700/700 [==============================] - 49s 71ms/step - loss: 0.4069 - accuracy: 0.8661 - val_loss: 0.3335 - val_accuracy: 0.8817
Epoch 4/10
700/700 [==============================] - 46s 66ms/step - loss: 0.3764 - accuracy: 0.8752 - val_loss: 0.3015 - val_accuracy: 0.8940
Epoch 5/10
700/700 [==============================] - 49s 70ms/step - loss: 0.3696 - accuracy: 0.8800 - val_loss: 0.2757 - val_accuracy: 0.9037
Epoch 6/10
700/700 [==============================] - 46s 66ms/step - loss: 0.3560 - accuracy: 0.8843 - val_loss: 0.2848 - val_accuracy: 0.8996
Epoch 7/10
700/700 [==============================] - 45s 65ms/step - loss: 0.3519 - accuracy: 0.8867 - val_loss: 0.2797 - val_accura

In [72]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 69s 90ms/step - loss: 0.7695 - accuracy: 0.7334 - val_loss: 0.3913 - val_accuracy: 0.8535
Epoch 2/10
744/744 [==============================] - 64s 86ms/step - loss: 0.4685 - accuracy: 0.8436 - val_loss: 0.3224 - val_accuracy: 0.8808
Epoch 3/10
744/744 [==============================] - 67s 90ms/step - loss: 0.4089 - accuracy: 0.8648 - val_loss: 0.3026 - val_accuracy: 0.8916
Epoch 4/10
744/744 [==============================] - 67s 90ms/step - loss: 0.3838 - accuracy: 0.8746 - val_loss: 0.3054 - val_accuracy: 0.8932
Epoch 5/10
744/744 [==============================] - 67s 90ms/step - loss: 0.3705 - accuracy: 0.8793 - val_loss: 0.3085 - val_accuracy: 0.8878
Epoch 6/10
744/744 [==============================] - 67s 90ms/step - loss: 0.3633 - accuracy: 0.8803 - val_loss: 0.3038 - val_accuracy: 0.8946
Epoch 7/10
744/744 [==============================] - 67s 90ms/step - loss: 0.3598 - accuracy: 0.8844 - val_loss: 0.3038 - val_accura

In [73]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 71s 88ms/step - loss: 0.7413 - accuracy: 0.7424 - val_loss: 0.3878 - val_accuracy: 0.8636
Epoch 2/10
788/788 [==============================] - 70s 88ms/step - loss: 0.4538 - accuracy: 0.8455 - val_loss: 0.3324 - val_accuracy: 0.8778
Epoch 3/10
788/788 [==============================] - 69s 88ms/step - loss: 0.4010 - accuracy: 0.8660 - val_loss: 0.3057 - val_accuracy: 0.8923
Epoch 4/10
788/788 [==============================] - 68s 86ms/step - loss: 0.3748 - accuracy: 0.8753 - val_loss: 0.3127 - val_accuracy: 0.8909
Epoch 5/10
788/788 [==============================] - 68s 87ms/step - loss: 0.3659 - accuracy: 0.8792 - val_loss: 0.2957 - val_accuracy: 0.8976
Epoch 6/10
788/788 [==============================] - 69s 88ms/step - loss: 0.3676 - accuracy: 0.8815 - val_loss: 0.3567 - val_accuracy: 0.8918
Epoch 7/10
788/788 [==============================] - 69s 88ms/step - loss: 0.3633 - accuracy: 0.8836 - val_loss: 0.3693 - val_accura

In [ ]:
time.sleep(400)

In [74]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 76s 90ms/step - loss: 0.7411 - accuracy: 0.7405 - val_loss: 0.3844 - val_accuracy: 0.8551
Epoch 2/10
832/832 [==============================] - 72s 87ms/step - loss: 0.4594 - accuracy: 0.8460 - val_loss: 0.3205 - val_accuracy: 0.8853
Epoch 3/10
832/832 [==============================] - 72s 87ms/step - loss: 0.4020 - accuracy: 0.8677 - val_loss: 0.3376 - val_accuracy: 0.8849
Epoch 4/10
832/832 [==============================] - 72s 87ms/step - loss: 0.3780 - accuracy: 0.8766 - val_loss: 0.3481 - val_accuracy: 0.8816
Epoch 5/10
832/832 [==============================] - 72s 87ms/step - loss: 0.3712 - accuracy: 0.8792 - val_loss: 0.3436 - val_accuracy: 0.8911
Epoch 6/10
832/832 [==============================] - 74s 89ms/step - loss: 0.3627 - accuracy: 0.8843 - val_loss: 0.3091 - val_accuracy: 0.8893
Epoch 7/10
832/832 [==============================] - 75s 90ms/step - loss: 0.3626 - accuracy: 0.8837 - val_loss: 0.3371 - val_accura

In [ ]:
time.sleep(400)

In [75]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 79s 88ms/step - loss: 0.7248 - accuracy: 0.7473 - val_loss: 0.3649 - val_accuracy: 0.8676
Epoch 2/10
875/875 [==============================] - 78s 89ms/step - loss: 0.4424 - accuracy: 0.8544 - val_loss: 0.3003 - val_accuracy: 0.8912
Epoch 3/10
875/875 [==============================] - 78s 89ms/step - loss: 0.3896 - accuracy: 0.8723 - val_loss: 0.3031 - val_accuracy: 0.8924
Epoch 4/10
875/875 [==============================] - 76s 87ms/step - loss: 0.3729 - accuracy: 0.8788 - val_loss: 0.2946 - val_accuracy: 0.8961
Epoch 5/10
875/875 [==============================] - 75s 86ms/step - loss: 0.3654 - accuracy: 0.8824 - val_loss: 0.2846 - val_accuracy: 0.8977
Epoch 6/10
875/875 [==============================] - 75s 85ms/step - loss: 0.3586 - accuracy: 0.8853 - val_loss: 0.2778 - val_accuracy: 0.9007
Epoch 7/10
875/875 [==============================] - 77s 88ms/step - loss: 0.3611 - accuracy: 0.8858 - val_loss: 0.3045 - val_accura

In [76]:
len(models_random)

20

In [77]:
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_random_e1"

i=12

for model in models_random[i:]:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_random_e1_19\assets
Model has been saved


In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [78]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [80]:
import gc
gc.collect()

24242

## Training guided by NC

In [82]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "D:/guided-retraining/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [83]:
nc_values.shape

(55998,)

In [84]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [85]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [90]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_nc.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [91]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 10s 207ms/step - loss: 0.9771 - accuracy: 0.7225 - val_loss: 2.9484 - val_accuracy: 0.1834
Epoch 2/10
44/44 [==============================] - 9s 205ms/step - loss: 0.5966 - accuracy: 0.8457 - val_loss: 2.7337 - val_accuracy: 0.2197
Epoch 3/10
44/44 [==============================] - 9s 198ms/step - loss: 0.5260 - accuracy: 0.8582 - val_loss: 2.4956 - val_accuracy: 0.2156
Epoch 4/10
44/44 [==============================] - 8s 194ms/step - loss: 0.4722 - accuracy: 0.8693 - val_loss: 2.5077 - val_accuracy: 0.2276
Epoch 5/10
44/44 [==============================] - 8s 193ms/step - loss: 0.4408 - accuracy: 0.8736 - val_loss: 2.3435 - val_accuracy: 0.2252
Epoch 6/10
44/44 [==============================] - 9s 197ms/step - loss: 0.4233 - accuracy: 0.8761 - val_loss: 2.9106 - val_accuracy: 0.2361
Epoch 7/10
44/44 [==============================] - 8s 193ms/step - loss: 0.3953 - accuracy: 0.8786 - val_loss: 2.2638 - val_accuracy: 0.2342
Epo

In [92]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 14s 139ms/step - loss: 0.9480 - accuracy: 0.7202 - val_loss: 2.6829 - val_accuracy: 0.3553
Epoch 2/10
88/88 [==============================] - 12s 135ms/step - loss: 0.5636 - accuracy: 0.8368 - val_loss: 2.7969 - val_accuracy: 0.3506
Epoch 3/10
88/88 [==============================] - 12s 135ms/step - loss: 0.4849 - accuracy: 0.8600 - val_loss: 2.4802 - val_accuracy: 0.3899
Epoch 4/10
88/88 [==============================] - 12s 139ms/step - loss: 0.4298 - accuracy: 0.8725 - val_loss: 2.5577 - val_accuracy: 0.4104
Epoch 5/10
88/88 [==============================] - 13s 143ms/step - loss: 0.3865 - accuracy: 0.8838 - val_loss: 2.8332 - val_accuracy: 0.4143
Epoch 6/10
88/88 [==============================] - 12s 132ms/step - loss: 0.3683 - accuracy: 0.8923 - val_loss: 2.1529 - val_accuracy: 0.4240
Epoch 7/10
88/88 [==============================] - 11s 120ms/step - loss: 0.3492 - accuracy: 0.8957 - val_loss: 2.8989 - val_accuracy: 0.41

In [93]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 17s 120ms/step - loss: 0.8583 - accuracy: 0.7501 - val_loss: 2.1843 - val_accuracy: 0.4253
Epoch 2/10
132/132 [==============================] - 16s 118ms/step - loss: 0.4762 - accuracy: 0.8705 - val_loss: 2.2112 - val_accuracy: 0.4868
Epoch 3/10
132/132 [==============================] - 15s 117ms/step - loss: 0.3990 - accuracy: 0.8900 - val_loss: 2.3643 - val_accuracy: 0.5601
Epoch 4/10
132/132 [==============================] - 15s 115ms/step - loss: 0.3463 - accuracy: 0.9029 - val_loss: 1.8761 - val_accuracy: 0.5784
Epoch 5/10
132/132 [==============================] - 16s 118ms/step - loss: 0.3168 - accuracy: 0.9118 - val_loss: 2.4389 - val_accuracy: 0.6059
Epoch 6/10
132/132 [==============================] - 15s 117ms/step - loss: 0.3062 - accuracy: 0.9151 - val_loss: 2.3951 - val_accuracy: 0.6071
Epoch 7/10
132/132 [==============================] - 15s 117ms/step - loss: 0.2736 - accuracy: 0.9250 - val_loss: 2.3926 - val_

In [94]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 20s 106ms/step - loss: 0.8070 - accuracy: 0.7565 - val_loss: 2.2828 - val_accuracy: 0.5090
Epoch 2/10
175/175 [==============================] - 18s 104ms/step - loss: 0.4484 - accuracy: 0.8756 - val_loss: 1.9367 - val_accuracy: 0.6516
Epoch 3/10
175/175 [==============================] - 18s 104ms/step - loss: 0.3704 - accuracy: 0.9001 - val_loss: 1.9511 - val_accuracy: 0.6996
Epoch 4/10
175/175 [==============================] - 19s 107ms/step - loss: 0.3257 - accuracy: 0.9129 - val_loss: 1.9800 - val_accuracy: 0.7241
Epoch 5/10
175/175 [==============================] - 18s 104ms/step - loss: 0.2977 - accuracy: 0.9194 - val_loss: 1.8426 - val_accuracy: 0.7221
Epoch 6/10
175/175 [==============================] - 18s 102ms/step - loss: 0.2747 - accuracy: 0.9274 - val_loss: 1.8320 - val_accuracy: 0.7282
Epoch 7/10
175/175 [==============================] - 18s 104ms/step - loss: 0.2557 - accuracy: 0.9307 - val_loss: 1.7351 - val_

In [95]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 26s 112ms/step - loss: 0.8142 - accuracy: 0.7518 - val_loss: 2.0669 - val_accuracy: 0.6498
Epoch 2/10
219/219 [==============================] - 24s 108ms/step - loss: 0.4129 - accuracy: 0.8846 - val_loss: 1.9900 - val_accuracy: 0.7169
Epoch 3/10
219/219 [==============================] - 23s 107ms/step - loss: 0.3275 - accuracy: 0.9114 - val_loss: 2.3065 - val_accuracy: 0.7220
Epoch 4/10
219/219 [==============================] - 23s 106ms/step - loss: 0.2957 - accuracy: 0.9203 - val_loss: 2.3855 - val_accuracy: 0.7267
Epoch 5/10
219/219 [==============================] - 23s 107ms/step - loss: 0.2669 - accuracy: 0.9291 - val_loss: 1.8780 - val_accuracy: 0.7330 
Epoch 6/10
219/219 [==============================] - 23s 104ms/step - loss: 0.2528 - accuracy: 0.9308 - val_loss: 1.4845 - val_accuracy: 0.7389
Epoch 7/10
219/219 [==============================] - 22s 101ms/step - loss: 0.2353 - accuracy: 0.9371 - val_loss: 2.0213 - val

In [96]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 28s 103ms/step - loss: 0.7770 - accuracy: 0.7561 - val_loss: 1.8881 - val_accuracy: 0.6827
Epoch 2/10
263/263 [==============================] - 25s 96ms/step - loss: 0.3877 - accuracy: 0.8892 - val_loss: 2.0738 - val_accuracy: 0.7106
Epoch 3/10
263/263 [==============================] - 27s 102ms/step - loss: 0.3081 - accuracy: 0.9158 - val_loss: 1.8955 - val_accuracy: 0.7330
Epoch 4/10
263/263 [==============================] - 32s 120ms/step - loss: 0.2767 - accuracy: 0.9246 - val_loss: 1.7315 - val_accuracy: 0.7392
Epoch 5/10
263/263 [==============================] - 27s 103ms/step - loss: 0.2467 - accuracy: 0.9355 - val_loss: 1.6940 - val_accuracy: 0.7443
Epoch 6/10
263/263 [==============================] - 26s 100ms/step - loss: 0.2251 - accuracy: 0.9412 - val_loss: 1.5940 - val_accuracy: 0.7456
Epoch 7/10
263/263 [==============================] - 26s 100ms/step - loss: 0.2151 - accuracy: 0.9421 - val_loss: 1.6104 - val_a

In [97]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 32s 98ms/step - loss: 0.7073 - accuracy: 0.7784 - val_loss: 2.4072 - val_accuracy: 0.7026
Epoch 2/10
307/307 [==============================] - 30s 99ms/step - loss: 0.3393 - accuracy: 0.9048 - val_loss: 2.6282 - val_accuracy: 0.7286
Epoch 3/10
307/307 [==============================] - 33s 107ms/step - loss: 0.2799 - accuracy: 0.9256 - val_loss: 2.4791 - val_accuracy: 0.7426
Epoch 4/10
307/307 [==============================] - 30s 97ms/step - loss: 0.2492 - accuracy: 0.9348 - val_loss: 2.3416 - val_accuracy: 0.7493
Epoch 5/10
307/307 [==============================] - 30s 98ms/step - loss: 0.2281 - accuracy: 0.9409 - val_loss: 2.0473 - val_accuracy: 0.7504
Epoch 6/10
307/307 [==============================] - 29s 95ms/step - loss: 0.2203 - accuracy: 0.9434 - val_loss: 2.2664 - val_accuracy: 0.7391
Epoch 7/10
307/307 [==============================] - 29s 96ms/step - loss: 0.2086 - accuracy: 0.9460 - val_loss: 1.5475 - val_accura

In [99]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 62s 176ms/step - loss: 0.6784 - accuracy: 0.7894 - val_loss: 2.0806 - val_accuracy: 0.7151
Epoch 2/10
350/350 [==============================] - 35s 101ms/step - loss: 0.3394 - accuracy: 0.9058 - val_loss: 1.9993 - val_accuracy: 0.7365
Epoch 3/10
350/350 [==============================] - 35s 101ms/step - loss: 0.2705 - accuracy: 0.9282 - val_loss: 2.5328 - val_accuracy: 0.7498
Epoch 4/10
350/350 [==============================] - 35s 99ms/step - loss: 0.2445 - accuracy: 0.9354 - val_loss: 1.4971 - val_accuracy: 0.7507
Epoch 5/10
350/350 [==============================] - 35s 100ms/step - loss: 0.2240 - accuracy: 0.9432 - val_loss: 2.9312 - val_accuracy: 0.7590
Epoch 6/10
350/350 [==============================] - 34s 96ms/step - loss: 0.2136 - accuracy: 0.9450 - val_loss: 1.4298 - val_accuracy: 0.7527
Epoch 7/10
350/350 [==============================] - 34s 96ms/step - loss: 0.2050 - accuracy: 0.9472 - val_loss: 1.4034 - val_acc

In [100]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 39s 97ms/step - loss: 0.6754 - accuracy: 0.7827 - val_loss: 2.2921 - val_accuracy: 0.7216
Epoch 2/10
394/394 [==============================] - 38s 97ms/step - loss: 0.3247 - accuracy: 0.9083 - val_loss: 1.8502 - val_accuracy: 0.7410
Epoch 3/10
394/394 [==============================] - 38s 96ms/step - loss: 0.2609 - accuracy: 0.9307 - val_loss: 1.6027 - val_accuracy: 0.7544
Epoch 4/10
394/394 [==============================] - 38s 96ms/step - loss: 0.2337 - accuracy: 0.9392 - val_loss: 0.9835 - val_accuracy: 0.7653
Epoch 5/10
394/394 [==============================] - 38s 96ms/step - loss: 0.2129 - accuracy: 0.9445 - val_loss: 1.8594 - val_accuracy: 0.7646
Epoch 6/10
394/394 [==============================] - 38s 96ms/step - loss: 0.2077 - accuracy: 0.9456 - val_loss: 1.4654 - val_accuracy: 0.7631
Epoch 7/10
394/394 [==============================] - 38s 96ms/step - loss: 0.2044 - accuracy: 0.9477 - val_loss: 1.1656 - val_accurac

In [101]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 45s 99ms/step - loss: 0.6725 - accuracy: 0.7885 - val_loss: 1.5828 - val_accuracy: 0.7275
Epoch 2/10
438/438 [==============================] - 43s 98ms/step - loss: 0.3094 - accuracy: 0.9149 - val_loss: 1.1094 - val_accuracy: 0.7454
Epoch 3/10
438/438 [==============================] - 43s 98ms/step - loss: 0.2547 - accuracy: 0.9335 - val_loss: 0.8390 - val_accuracy: 0.7634
Epoch 4/10
438/438 [==============================] - 43s 98ms/step - loss: 0.2296 - accuracy: 0.9410 - val_loss: 0.8618 - val_accuracy: 0.7717
Epoch 5/10
438/438 [==============================] - 43s 98ms/step - loss: 0.2103 - accuracy: 0.9445 - val_loss: 0.9518 - val_accuracy: 0.7735
Epoch 6/10
438/438 [==============================] - 43s 97ms/step - loss: 0.2091 - accuracy: 0.9467 - val_loss: 0.9112 - val_accuracy: 0.7731
Epoch 7/10
438/438 [==============================] - 43s 98ms/step - loss: 0.1993 - accuracy: 0.9492 - val_loss: 1.0023 - val_accurac

In [102]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 48s 97ms/step - loss: 0.6361 - accuracy: 0.7950 - val_loss: 0.6216 - val_accuracy: 0.8204
Epoch 2/10
482/482 [==============================] - 47s 97ms/step - loss: 0.3021 - accuracy: 0.9150 - val_loss: 0.5051 - val_accuracy: 0.8482
Epoch 3/10
482/482 [==============================] - 47s 97ms/step - loss: 0.2531 - accuracy: 0.9316 - val_loss: 0.4824 - val_accuracy: 0.8571
Epoch 4/10
482/482 [==============================] - 47s 97ms/step - loss: 0.2247 - accuracy: 0.9402 - val_loss: 0.5070 - val_accuracy: 0.8547
Epoch 5/10
482/482 [==============================] - 46s 96ms/step - loss: 0.2139 - accuracy: 0.9452 - val_loss: 0.5100 - val_accuracy: 0.8671
Epoch 6/10
482/482 [==============================] - 47s 96ms/step - loss: 0.2079 - accuracy: 0.9455 - val_loss: 0.5914 - val_accuracy: 0.8617
Epoch 7/10
482/482 [==============================] - 47s 97ms/step - loss: 0.1985 - accuracy: 0.9483 - val_loss: 0.5364 - val_accura

In [103]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 49s 91ms/step - loss: 0.6247 - accuracy: 0.7986 - val_loss: 0.5100 - val_accuracy: 0.8354
Epoch 2/10
525/525 [==============================] - 47s 89ms/step - loss: 0.3016 - accuracy: 0.9152 - val_loss: 0.4905 - val_accuracy: 0.8536
Epoch 3/10
525/525 [==============================] - 47s 90ms/step - loss: 0.2447 - accuracy: 0.9330 - val_loss: 0.4590 - val_accuracy: 0.8664
Epoch 4/10
525/525 [==============================] - 146s 279ms/step - loss: 0.2253 - accuracy: 0.9385 - val_loss: 0.4412 - val_accuracy: 0.8696
Epoch 5/10
525/525 [==============================] - 44s 84ms/step - loss: 0.2160 - accuracy: 0.9443 - val_loss: 0.4069 - val_accuracy: 0.8768
Epoch 6/10
525/525 [==============================] - 38s 72ms/step - loss: 0.2053 - accuracy: 0.9457 - val_loss: 0.4706 - val_accuracy: 0.8652
Epoch 7/10
525/525 [==============================] - 35s 68ms/step - loss: 0.2080 - accuracy: 0.9466 - val_loss: 0.4838 - val_accu

In [104]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 47s 81ms/step - loss: 0.5996 - accuracy: 0.8072 - val_loss: 0.5561 - val_accuracy: 0.8334
Epoch 2/10
569/569 [==============================] - 37s 64ms/step - loss: 0.2982 - accuracy: 0.9167 - val_loss: 0.4371 - val_accuracy: 0.8597
Epoch 3/10
569/569 [==============================] - 36s 63ms/step - loss: 0.2467 - accuracy: 0.9340 - val_loss: 0.4463 - val_accuracy: 0.8709
Epoch 4/10
569/569 [==============================] - 35s 62ms/step - loss: 0.2264 - accuracy: 0.9394 - val_loss: 0.4361 - val_accuracy: 0.8709
Epoch 5/10
569/569 [==============================] - 35s 62ms/step - loss: 0.2165 - accuracy: 0.9429 - val_loss: 0.3996 - val_accuracy: 0.8779
Epoch 6/10
569/569 [==============================] - 36s 62ms/step - loss: 0.2071 - accuracy: 0.9463 - val_loss: 0.4938 - val_accuracy: 0.8729
Epoch 7/10
569/569 [==============================] - 36s 64ms/step - loss: 0.2053 - accuracy: 0.9470 - val_loss: 0.3782 - val_accura

In [105]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 40s 64ms/step - loss: 0.6240 - accuracy: 0.7973 - val_loss: 0.4709 - val_accuracy: 0.8453
Epoch 2/10
613/613 [==============================] - 38s 62ms/step - loss: 0.3128 - accuracy: 0.9115 - val_loss: 0.4156 - val_accuracy: 0.8676
Epoch 3/10
613/613 [==============================] - 39s 63ms/step - loss: 0.2590 - accuracy: 0.9290 - val_loss: 0.4355 - val_accuracy: 0.8690
Epoch 4/10
613/613 [==============================] - 39s 64ms/step - loss: 0.2379 - accuracy: 0.9357 - val_loss: 0.4077 - val_accuracy: 0.8768
Epoch 5/10
613/613 [==============================] - 40s 65ms/step - loss: 0.2192 - accuracy: 0.9398 - val_loss: 0.3686 - val_accuracy: 0.8809
Epoch 6/10
613/613 [==============================] - 42s 69ms/step - loss: 0.2160 - accuracy: 0.9409 - val_loss: 0.4197 - val_accuracy: 0.8819
Epoch 7/10
613/613 [==============================] - 41s 67ms/step - loss: 0.2176 - accuracy: 0.9438 - val_loss: 0.4499 - val_accura

In [106]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 46s 68ms/step - loss: 0.6202 - accuracy: 0.8008 - val_loss: 0.4868 - val_accuracy: 0.8407
Epoch 2/10
657/657 [==============================] - 45s 68ms/step - loss: 0.3125 - accuracy: 0.9100 - val_loss: 0.4580 - val_accuracy: 0.8666
Epoch 3/10
657/657 [==============================] - 44s 67ms/step - loss: 0.2690 - accuracy: 0.9257 - val_loss: 0.3900 - val_accuracy: 0.8710
Epoch 4/10
657/657 [==============================] - 44s 67ms/step - loss: 0.2426 - accuracy: 0.9334 - val_loss: 0.4985 - val_accuracy: 0.8704
Epoch 5/10
657/657 [==============================] - 43s 66ms/step - loss: 0.2319 - accuracy: 0.9380 - val_loss: 0.5503 - val_accuracy: 0.8760
Epoch 6/10
657/657 [==============================] - 43s 66ms/step - loss: 0.2320 - accuracy: 0.9374 - val_loss: 0.4424 - val_accuracy: 0.8783
Epoch 7/10
657/657 [==============================] - 43s 65ms/step - loss: 0.2248 - accuracy: 0.9390 - val_loss: 0.5606 - val_accura

In [107]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 50s 70ms/step - loss: 0.5905 - accuracy: 0.8092 - val_loss: 0.4198 - val_accuracy: 0.8585
Epoch 2/10
700/700 [==============================] - 48s 69ms/step - loss: 0.3113 - accuracy: 0.9096 - val_loss: 0.4170 - val_accuracy: 0.8715
Epoch 3/10
700/700 [==============================] - 47s 68ms/step - loss: 0.2669 - accuracy: 0.9263 - val_loss: 0.3738 - val_accuracy: 0.8799
Epoch 4/10
700/700 [==============================] - 47s 68ms/step - loss: 0.2444 - accuracy: 0.9319 - val_loss: 0.3619 - val_accuracy: 0.8832
Epoch 5/10
700/700 [==============================] - 49s 70ms/step - loss: 0.2333 - accuracy: 0.9352 - val_loss: 0.3881 - val_accuracy: 0.8859
Epoch 6/10
700/700 [==============================] - 47s 67ms/step - loss: 0.2334 - accuracy: 0.9361 - val_loss: 0.4251 - val_accuracy: 0.8873
Epoch 7/10
700/700 [==============================] - 48s 68ms/step - loss: 0.2342 - accuracy: 0.9362 - val_loss: 0.4089 - val_accura

In [ ]:
time.sleep(400)

In [108]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 52s 68ms/step - loss: 0.6276 - accuracy: 0.7914 - val_loss: 0.4122 - val_accuracy: 0.8533
Epoch 2/10
744/744 [==============================] - 50s 67ms/step - loss: 0.3414 - accuracy: 0.8971 - val_loss: 0.3573 - val_accuracy: 0.8764
Epoch 3/10
744/744 [==============================] - 49s 66ms/step - loss: 0.2948 - accuracy: 0.9136 - val_loss: 0.3192 - val_accuracy: 0.8914
Epoch 4/10
744/744 [==============================] - 50s 68ms/step - loss: 0.2686 - accuracy: 0.9225 - val_loss: 0.3931 - val_accuracy: 0.8867
Epoch 5/10
744/744 [==============================] - 49s 65ms/step - loss: 0.2654 - accuracy: 0.9248 - val_loss: 0.3463 - val_accuracy: 0.8933
Epoch 6/10
744/744 [==============================] - 49s 66ms/step - loss: 0.2575 - accuracy: 0.9273 - val_loss: 0.3734 - val_accuracy: 0.8880
Epoch 7/10
744/744 [==============================] - 49s 66ms/step - loss: 0.2630 - accuracy: 0.9279 - val_loss: 0.3435 - val_accura

In [ ]:
time.sleep(400)

In [109]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 54s 67ms/step - loss: 0.6610 - accuracy: 0.7794 - val_loss: 0.3961 - val_accuracy: 0.8636
Epoch 2/10
788/788 [==============================] - 52s 66ms/step - loss: 0.3725 - accuracy: 0.8838 - val_loss: 0.3494 - val_accuracy: 0.8786
Epoch 3/10
788/788 [==============================] - 53s 67ms/step - loss: 0.3235 - accuracy: 0.9029 - val_loss: 0.3188 - val_accuracy: 0.8903
Epoch 4/10
788/788 [==============================] - 52s 66ms/step - loss: 0.3011 - accuracy: 0.9087 - val_loss: 0.3238 - val_accuracy: 0.8881
Epoch 5/10
788/788 [==============================] - 52s 66ms/step - loss: 0.2897 - accuracy: 0.9144 - val_loss: 0.3307 - val_accuracy: 0.8921
Epoch 6/10
788/788 [==============================] - 52s 66ms/step - loss: 0.2911 - accuracy: 0.9144 - val_loss: 0.3152 - val_accuracy: 0.8941
Epoch 7/10
788/788 [==============================] - 51s 65ms/step - loss: 0.2914 - accuracy: 0.9166 - val_loss: 0.2998 - val_accura

In [ ]:
time.sleep(400)

In [110]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 57s 68ms/step - loss: 0.6946 - accuracy: 0.7583 - val_loss: 0.3816 - val_accuracy: 0.8627
Epoch 2/10
832/832 [==============================] - 56s 67ms/step - loss: 0.4116 - accuracy: 0.8675 - val_loss: 0.3220 - val_accuracy: 0.8865
Epoch 3/10
832/832 [==============================] - 56s 67ms/step - loss: 0.3594 - accuracy: 0.8846 - val_loss: 0.3169 - val_accuracy: 0.8853
Epoch 4/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3397 - accuracy: 0.8941 - val_loss: 0.3068 - val_accuracy: 0.8936
Epoch 5/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3282 - accuracy: 0.8965 - val_loss: 0.3592 - val_accuracy: 0.8946
Epoch 6/10
832/832 [==============================] - 56s 67ms/step - loss: 0.3289 - accuracy: 0.8982 - val_loss: 0.2951 - val_accuracy: 0.8974
Epoch 7/10
832/832 [==============================] - 55s 66ms/step - loss: 0.3248 - accuracy: 0.9001 - val_loss: 0.3050 - val_accura

In [ ]:
time.sleep(400)

In [111]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 67s 75ms/step - loss: 0.7234 - accuracy: 0.7427 - val_loss: 0.3851 - val_accuracy: 0.8583
Epoch 2/10
875/875 [==============================] - 64s 73ms/step - loss: 0.4522 - accuracy: 0.8494 - val_loss: 0.3280 - val_accuracy: 0.8789
Epoch 3/10
875/875 [==============================] - 64s 73ms/step - loss: 0.3998 - accuracy: 0.8690 - val_loss: 0.2930 - val_accuracy: 0.8918
Epoch 4/10
875/875 [==============================] - 64s 73ms/step - loss: 0.3790 - accuracy: 0.8758 - val_loss: 0.2863 - val_accuracy: 0.8984
Epoch 5/10
875/875 [==============================] - 64s 73ms/step - loss: 0.3715 - accuracy: 0.8795 - val_loss: 0.3073 - val_accuracy: 0.8971
Epoch 6/10
875/875 [==============================] - 63s 73ms/step - loss: 0.3695 - accuracy: 0.8819 - val_loss: 0.2908 - val_accuracy: 0.9011
Epoch 7/10
875/875 [==============================] - 64s 73ms/step - loss: 0.3665 - accuracy: 0.8827 - val_loss: 0.3077 - val_accura

In [112]:
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_may_nc_e1"

i=7

for model in models_nc[i:]:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_8\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_9\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C1/fashion_model_c1_may_nc_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: 

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [113]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [114]:
gc.collect()

90765